In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

##    Description    Functions to manage SDFiles, pandas Dataframes ...
##                   Applicability Domain analysis
##                   
##    Authors:       Kevin Pinto Gil (kevin.pinto@upf.edu)
##                   Manuel Pastor (manuel.pastor@upf.edu)
##
##    Copyright 2018 Manuel Pastor
##
##    This file is part of PhiTools
##
##    PhiTools is free software: you can redistribute it and/or modify
##    it under the terms of the GNU General Public License as published by
##    the Free Software Foundation version 3.
##
##    PhiTools is distributed in the hope that it will be useful,
##    but WITHOUT ANY WARRANTY; without even the implied warranty of
##    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
##    GNU General Public License for more details.
##
##    You should have received a copy of the GNU General Public License
##    along with PhiTools.  If not, see <http://www.gnu.org/licenses/>

# 1. Importing libraries

In [2]:
### System libraries

import sys
import os
import getopt
import re
import shutil

### General libraries

import pandas as pd
import numpy as np
from math import * #math commands will be available every time you start an interactive session

## RDkit libraries

from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, AllChem, Descriptors, Crippen, DataStructs

### LoadSDF into Pandas Dataframe without removing Hs
from __future__ import print_function 

from base64 import b64encode 
import sys 
import types 
import numpy as np 
from rdkit import Chem 
from rdkit import DataStructs 
from rdkit.Chem import AllChem 
from rdkit.Chem import Draw 
from rdkit.Chem.Draw import rdMolDraw2D 
from rdkit.Chem import SDWriter 
from rdkit.Chem import rdchem 
from rdkit.Chem.Scaffolds import MurckoScaffold 
from rdkit.six import BytesIO, string_types, PY3 

### Standardise a molecule libraries

from standardiser import process_smiles as ps
from standardiser import neutralise
from molvs import tautomer
from phitools import moleculeHelper as mh

### Scikit learn libraries

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


### Graphical libraries

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl ## to come back to matplotlib default parameters
import seaborn as sns

# mpl.use('Agg') ## ERROOOOOOOOOOOOOOOOOOOOR

from pylab import *
%matplotlib inline

## Dataframe visualization part

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 4000

## Ignore Warnings 

import warnings
warnings.filterwarnings('ignore')


*** Could not find EPA module. Will use only the CACTVS web service to resolve CAS number structures. ***



# 2. Normalization

## 2.1. Standardize molecules 
- using Standardiser (F. Atkinson) plus Bet Gregori Modifications

In [1]:
def getNormMol (smiles, neutralize=True):
    '''
    Info
    ----
    This function it is going to:
       - use standardiser for removing salts, mixtures, metal ions.
       - use molvs for canonicalize the smile keeping tautomeric form
         and decouple metal covalent bonds.
       - use standardiser to neutralize smiles.
    
    Parameters
    ----------
    
    smiles = 'smiles' ## smile column name
    df = df ## dataframe name
    neutralize = True ## True (default Value) if one wants to neutralize the smile, otherwise False. 
    
    Return
    ------
    
    pandas series with ( inchikey, standard smiles, Standardisation information )
    
    Example
    -------
    
    One can run the function like this and join 3 new colulmns to your dataframe
    containing the standardization results. 
    
           df[['parent_std_inkey', 'std_smiles', 'info_standardization']] = DF.apply(
            lambda row: getNormMol(row['SMILES'], df, NO), axis=1)
    '''
    canon = tautomer.TautomerCanonicalizer()
    mol = Chem.MolFromSmiles(smiles)
    stdD = ps.std(mol)
    if len(stdD) > 1:
        results = pd.Series(('NA', 'NA', 'Not Passed: There is more than 1 molecule'))
    else:
        std_smiles = list(stdD.keys()).pop()
        (mol, ismetal, passed, errmessage) = stdD[std_smiles]
        if ismetal:
            results = pd.Series(('METAL', 'METAL', 'Not passed: There is Metal Ion'))
        elif not passed:
            mol = canon.canonicalize(mol)
            if neutralize == True:
                mol = neutralise.run(mol)
            std_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
            inchi = Chem.MolToInchi(mol)
            inchikey = Chem.InchiToInchiKey(inchi)
            ns_inchi = Chem.MolToInchi(mol, options='/FixedH')
            ns_inchikey = Chem.InchiToInchiKey(ns_inchi)
            results = pd.Series((inchikey, std_smiles,'Smile Not Standardised'))

        else:
            mol = canon.canonicalize(mol)
            if neutralize == True:
                mol = neutralise.run(mol)
            std_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
            inchi = Chem.MolToInchi(mol)
            inchikey = Chem.InchiToInchiKey(inchi)
            ns_inchi = Chem.MolToInchi(mol, options='/FixedH')
            ns_inchikey = Chem.InchiToInchiKey(ns_inchi)
            results = pd.Series((inchikey, std_smiles, 'Smile Standardised'))
    return results

In [ ]:
def getStdMol(df, molcol, outName):

    '''
        This functions allows one to standardize molecules following F. Atkinson protocol
        plus Bet Gregori Modifications:
        
        Input parameters:
            inDF = cmrDF        ## pandas Dataframe containing column
            molcol = 'mol'      ## molecule column
            outName =  'cmr'    ## basename to name out files
        
        Outputs:
            File-normalized.sdf ## including molecules normalized
            File-excluded.sdf   ## including molecules excluded
            incDF ## pandasDataframe with included molecules
            excDF ## pandasDataframe with excluded molecules
        
        e.g. incDF, excDF = getStandadisedMol(inSDF,molcol, outName)
        
        NOTE: If a molecule contains a metal ion coupled to a molecule, this standardiser is
        using molVS to decouple the metal ion and rebuild the other part of the molecule 
        to be standardised. So It is probable that the Total final molecule number increase due
        that. 

    '''  
   
    excDF = pd.DataFrame([])
    incDF = pd.DataFrame([])
    
    df['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(df))] ## creating phiID

    for i, row in df.iterrows():
        mol = row[molcol]
        ## Checking Empty molecules:
        ##       not sure if this is well done with pandasDataframe
        ##       when loading with pandasTools, empty molecules are Eliminated I think
        if mol is None:
            passed = 'Failed'
            error = 'Creating empty molecule'
            psmiles = ''
            pinchi = ''
            pinkey = ''
            ismetal = ''
            
            molblock = suppl.GetItemText(i).rstrip()[:-4]
            mol = Chem.MolFromMolBlock(molblock)

            emptydic = {'phiID': phiID, 'info_std': passed, molcol: mol,
                        'parent_mol': stdmol,'parent_smiles':psmiles,
                        'parent_inchi': pinchi,'parent_inkey': pinkey,
                        'error':error, 'is_metal': ismetal}
            emptyDF = pd.DataFrame([emptydic])
            excDF = excDF.append(emtpyDF)
            continue

        ### Standardising molecules:
        phiID = df['phiID'][i] ### phiID for merging data 
        
        stdD = ps.std(mol) ### this standardise the molecule giving a dictionary
        
        for psmiles in stdD:
            stdmol, ismetal, passed, error = stdD[psmiles]
            stdmol = Chem.MolFromSmiles(psmiles)
            
            pinchi = Chem.MolToInchi(stdmol)
            pinkey = Chem.InchiToInchiKey(pinchi)
           
            ## Metal Ion condition (Bet standardiser include metals for inditex, but I dont want them)
            if ismetal == True:
                ismetal = 'Metal ion Decoupled'

                passed = False ### converting passed function True to False to avoid including metals
            else:
                ismetal = ''
            
            ## Standardise section
            if passed == True:

                passed = 'Standardised'
                
                ## Incoporating new columns with standardise information

                tempdic = {'phiID': phiID, 'info_std': passed, molcol: mol,
                           'parent_mol': stdmol,'parent_smiles':psmiles,
                           'parent_inchi': pinchi,'parent_inkey': pinkey,
                           'is_metal': ismetal
                          }
                tempDF = pd.DataFrame([tempdic])

                incDF = incDF.append(tempDF)
           
            ### Excluding molecules section
            else:
               
                if not error:
                    error = 'No error reported'
                    
                passed = 'Failed'
                
                ## Incoporating new columns with error information

                tempdic = {'phiID': phiID,'info_std': passed, molcol: mol,
                           'parent_mol': stdmol,'parent_smiles':psmiles,
                           'parent_inchi': pinchi,'parent_inkey': pinkey,
                           'error':error, 'is_metal': ismetal
                          }
                tempDF = pd.DataFrame([tempdic])
                excDF = excDF.append(tempDF)

    ### removing molcol from main dataframe to avoid duplicate columns            
    df = df.drop(columns=[molcol])            
    
    ### Merging data from main Dataframe  with new Standardise info in incDF dataframe
    if not incDF.empty:
        incDF = incDF.reset_index(drop=True) ## reseting index
        incDF = incDF[['phiID', molcol, 'parent_mol','parent_smiles', 'parent_inchi',
                       'parent_inkey', 'info_std', 'is_metal']]


        includedDF = pd.merge(incDF, df, left_on='phiID', right_on='phiID', how='inner')
        
#         ### Writing and SDF file with Standardised molecules
#         indf = includedDF.copy()
#         del indf[molcol]
#         writeSDFfromPandasDF(indf, outName+'-included.sdf', 'parent_mol', 
#                              props=list(indf.columns))
    else: 
        print('No included molecules has been found')
        pass

    ### Merging data from main Dataframe  with NO standardise molecules info in excDF dataframe
    
    if not excDF.empty: 
        excDF = excDF.reset_index(drop=True) ## reseting index
        excDF = excDF[['phiID', molcol, 'parent_mol','parent_smiles', 'parent_inchi',
                       'parent_inkey', 'info_std','error',  'is_metal']]
        excludedDF = pd.merge(excDF, df, left_on='phiID', right_on='phiID', how='inner')

#         ### Writing and SDF file with Standardised molecules
#         exdf = excludedDF.copy()
#         del exdf[molcol]
#         writeSDFfromPandasDF(exdf, outName+'-excluded.sdf', 'parent_mol',
#                              props=list(exdf.columns) )
    else:
        excludedDF = pd.DataFrame([])
        print('No excluded molecules has been found')
        pass
    
    return ( includedDF, excludedDF )

## 2.2. Checking Duplicates

In [ ]:
def getDupSingleDF(df, colname):
    
    '''
    
    This function checks if there is any duplicate molecule in both 
    standardised and excluded dataframes. It is highly recommended to checked
    this using as a colname the Inchikey.

    Input parameters:
        df = inditexDF       ## Original dataframe gave as a input DF in getStdMol
        colname = 'inchikey' ## molecule column to be used to check duplicates
        
    Output: 
        In console it will be printed something like this:
            1. duplicates checking section:
            The number of inchikey duplicates in are  = 0
        
        There will be returned two DF with duplicated molecules, the included and the excluded ones. 
    
    e.g. duplist = getDuplicatesDF(inditexDF, 'inchikey')
    '''
#     df['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(df))]

    if not df.empty:
    
        duplist = df[df[colname].duplicated()][colname]
#         print ('\033[1m' +'1. Duplicates checking section:'+'\033[0m')
        print ('\nOriginal DF molecules = '+ str(len(df)))
        print ('\nThe number of '+colname+' duplicates in are  = '+ str(duplist.count()))
        print ('\nThe '+ colname + 'duplicates list is: \n'+str(np.array(duplist)))

        ## This dataframe contains duplicated molecules with metal ion decoupled
        dupDF = df[df[colname].isin(duplist)]
#         dupDF = dupDF.sort_values(by=[colname, 'phiID'], ascending=[True, True])

    else:
        duplist = []
        print('There is no included molecules')
       
    return (duplist)

In [ ]:
def getDuplicatesDF(df, incDF, excDF, colname):
    
    '''
    
    This function checks if there is any duplicate molecule in both 
    standardised and excluded dataframes. It is highly recommended to checked
    this using as a colname the Inchikey.

    Input parameters:
        df = inditexDF       ## Original dataframe gave as a input DF in getStdMol
        incDF = incDF        ## DF containing standardised molecules
        excDF = excDF        ## DF containing excluded molecules by getStdMol function
        colname = 'inchikey' ## molecule column to be used to check duplicates
        
    Output: 
        In console it will be printed something like this:
            1. STANDARDISED duplicates checking section:
            The number of inchikey duplicates in are  = 0
            All Duplicated compounds containing metal ion index = []

            2. EXCLUDED duplicates checking section:
            The number of inchikey duplicates are  = 5
            All Duplicated compounds containing metal ion index = Int64Index([6, 70, 108, 166, 189], dtype='int64')

            3. Checking Molecules from Original dataframe coincide with standardised and excluded total molecules:
            ALL = 1113
            Standardised Molecules = 919
            Excluded Molecules = 199
            standardised + Excluded Molecules = 1118
        
        There will be returned two list with duplicated molecules, the included and the excluded ones. 
    
    e.g. dupinclist, dupexclist = getDuplicatesDF(inditexDF, incIndDF, excIndDF, 'inchikey')
    '''
    ### Included Section
    if not incDF.empty:
    
        dupinclist = incDF[incDF[colname].duplicated()][colname]
        print ('\033[1m' +'1. STANDARDISED duplicates checking section:'+'\033[0m')
        print ('\nThe number of '+colname+' duplicates in are  = '+ str(dupinclist.count()))
        print ('\nThe '+ colname + 'duplicates list is: \n'+str(np.array(dupinclist)))

        ## This dataframe contains duplicated molecules with metal ion decoupled
        dupincDF = incDF[incDF[colname].isin(dupinclist)]
        dupincDF = dupincDF.sort_values(by=['parent_inkey', 'phiID'], ascending=[True, True])

    #     ### Everything than contains metal ion decoupled

    #     metincID = incDF[incDF['is_metal'] == 'Metal ion Decoupled']
    #     print ('All compounds containing metal ion index = ' + (str(np.array(metincID.index))))

        ### Giving us only duplicates that contain metal ions

        metincIDdup = dupincDF[dupincDF['is_metal'] == 'Metal ion Decoupled']
        print ('\nAll Duplicated compounds containing metal ion index = ' + (str(np.array(metincIDdup.index)))) 
   
    else:
        dupinclist = []
        print('There is no included molecules')
    
    ### Excluded section
    if not excDF.empty:
    
        print ('\033[1m' + '\n2. EXCLUDED duplicates checking section:'+'\033[0m')
        dupexclist = excDF[excDF[colname].duplicated()][colname]
        print ('\nThe number of '+colname+' duplicates are  = '+ str(dupexclist.count()))
        print ('\nThe '+ colname + 'duplicates list is: \n'+str(np.array(dupexclist)))

        ## This dataframe contains duplicated molecules with metal ion decoupled
        dupexcDF = excDF[excDF[colname].isin(dupexclist)]
        dupexcDF = dupexcDF.sort_values(by=['parent_inkey', 'phiID'], ascending=[True, True])

    #     ### Everything than contains metal ion decoupled

    #     metexcID = excDF[excDF['is_metal'] == 'Metal ion Decoupled']
    #     print ('All compounds containing metal ion index = ' + (str(metexcID.index))) 

        ### Giving us only duplicates that contain metal ions

        metexcIDdup = dupexcDF[dupexcDF['is_metal'] == 'Metal ion Decoupled']
        print ('\nAll Duplicated compounds containing metal ion index = ' + (str(metexcIDdup.index))) 
        
    else:
        dupexclist = []
        print('There is no excluded molecules')

    ### Checking if everything is OK:
    
    print ('\033[1m' +'\n3. Final Checking:'+'\033[0m')
    print ('\nOriginal DF molecules = '+ str(len(df)))
    print ('\nStandardised Molecules = '+ str(len(incDF)))
    print ('Excluded Molecules = '+ str(len(excDF)))
    print ('\nStandardised + Excluded Molecules = '+ str(len(incDF)+len(excDF)))
    
    return (dupinclist, dupexclist)

In [ ]:
def findDupsGetNstdInkey(idf, smicol, inkeycol):
    
    '''
    Calculating Non-Standard Inchikey containing same Standard inchikey but they are Tautomers
    and changing the inchikeycolumn with Non-standard inchikeys 
    
    idf = df ### dataframe name
    smicol = 'std_smiles' ### smiles column name
    inkeycol = 'inchi_key' ### inchi key column name 
    
    e.g. findDupsGetNstdInkey(df, 'std_smiles', 'inchi_key')
    '''
    
    df = idf.copy()
    print('\033[1m \nChecking if there is any duplicate\033[0m')
    duplist1 = getDupSingleDF(df, inkeycol)
    print
    if len(duplist) > 0 :
        df.loc[df[inkeycol].isin(duplist1), inkeycol] = [addNonStdInchi(x) for x in df.loc[df[inkeycol].isin(duplist1), smicol]]
        print('\033[1m \nChecking duplicates after Non-Std-Inkey calculation\033[0m')
        duplist2 = getDupSingleDF(df, inkeycol)
    else:
        print('There is no duplicates')
    return (df, duplist1)

## 2.3. Dropping Dupplicates (keeping different info in list) and curate columns:

In [ ]:
def dropDupMols(df, colname, molcol= None):
    '''
    Explanation
    -----------
    
    After using getStdMol function, one needs to check duplicates with getDuplicatesDF
    function. After knowing how many duplicates you have, one should keep 1 molecule but
    keep the whole information of the other duplicate molecules. 
    This function allows one to obtain a dataframe where the duplicated molecule contains
    the information in a list in every column separated by comma. 
    
    Parameters
    ----------
    
    df = etoxDF 
              ## obtained after using getStdMol, but also after checking Excluded molecules
    colname = 'parent_inkey' 
              ## colname to groupby
    molcol = 'MolProt'
              ## By default is None, if there is molecule column defined, we will keep
              the first one. 
    
    Example
    -------        
    
    etoxNoDupDF = dropDupMols(etoxDF,'parent_inkey')
    
    Returns
    -------
    
    Dataframe with NO duplicates.
    Cells where non unique value was found, the value will be joined separated by ', ',
    such as "ROCHE_PC_RO0052807, AZ_GGA_200002374"
         
    '''
#     print (molcol)
    
    if not df.empty:
#         df1 = df.groupby([colname]).aggregate(lambda x : set(x))
#         cols = df.columns.drop(colname)
#         df1.reset_index(level=0, inplace=True)
#         df1[cols] = df1[cols].applymap(list)
        if molcol != None:
            ### Molecule Part
            df1 = df[[colname, molcol]].groupby([colname]).aggregate(lambda x : set(x))
            df1[molcol] = df1[molcol].apply(list)
            df1[molcol] = [x[0] for x in df1[molcol]]
            
            ### Rest of dataframe part
            cols = df.columns.drop(molcol)
            df2 = df[cols].groupby([colname]).aggregate(lambda x : ", ".join(map(str, set(x))))
            df1.reset_index(level=0, inplace=True)
            df2.reset_index(level=0, inplace=True)
            
            ### Joining molecule and rest of dataframe
            df3 = pd.merge(df2, df1 , left_on=colname, right_on=colname, how='inner')
            return(df3)
    
        else:
#             df1 = df.groupby([colname]).aggregate(lambda x : ", ".join(map(repr, set(x))))
            df1 = df.groupby([colname]).aggregate(lambda x : ", ".join(map(str, set(x))))
            df1.reset_index(level=0, inplace=True)
            return (df1)
            
#         return (df1)
        
    else:
        print('Dataframe is empty')
        df1 = pd.DataFrame([])
        return (df1)

In [ ]:
def curateCol(dfname, colname, t):
    '''
    This function allows one to curate columns from dataframe after using 
    my dropDupMols function.
    
    dfname = df   ### dataframe name
    colname = 'molecular_weight' ### column name to be curated
    t = float ### you need to choose between float or int or str
              ### str: it's going to keep the first item and erase the other ones
              ### int: convert string to integer and calcule the mean value. If any nan value, eliminate them.
              ### float: convert string to float and calcule the mean value. If any nan value, eliminate them.
    
    Returns a dataframe with the column given curated.
    
    e.g. curatedDF = curate(dropdupDF, 'std_smiles', str)
    '''
    df = dfname.copy()
    if t == float:
#         print ('I am a fucking float')
        for x, y in zip(df[colname], df.index):
            if ', ' in x:
                x = np.array(x.split(', ')).astype(np.float)
                if True in np.isnan(x):
                    x = x[~np.isnan(x)] 
                    if len(x) == 0:
                        x = nan
                        df.loc[df.index.isin([y]), colname] = x
                        
                    else:
                        x = np.mean(x)
                        df.loc[df.index.isin([y]), colname] = x
                        continue
                else:
                    x = np.mean(x)
                    df.loc[df.index.isin([y]), colname] = x
    elif t == int:
#         print ('I am a fucking int')
        for x, y in zip(df[colname], df.index):
            if ', ' in x:
                x = np.array(x.split(', ')).astype(np.int)
                if True in np.isnan(x):
                    x = x[~np.isnan(x)] 
                    if len(x) == 0:
                        x = nan
                        df.loc[df.index.isin([y]), colname] = x   
                    else:
                        x = np.mean(x)
                        df.loc[df.index.isin([y]), colname] = x
                        continue
                else:
                    x = np.mean(x)
                    df.loc[df.index.isin([y]), colname] = x
    elif t == str:
#         print ('This is only used for compounds with same stdinkey and different smile')
        for x, y in zip(df[colname], df.index):
            if ', ' in x:
                x = np.array(x.split(', '))
#                 print ( x, 'is changed for ' , x[0], 'with an index of ', y)
                x = x[0]
                df.loc[df.index.isin([y]), colname] = x
            else:
                df.loc[df.index.isin([y]), colname] = x
                continue

#     elif t == 'mol':
# #         print ('This is only used for compounds with same stdinkey and different smile')
#         for x, y in zip(df[colname], df.index):
#             if ', ' in x:
#                 x = np.array(x.split(', '))
#                 x = x[0]
#                 df.loc[df.index.isin([y]), colname] = x
#             else:
#                 df.loc[df.index.isin([y]), colname] = x
#                 continue
    return (df)

## 2.4. Dropping rows giving a list 
e.g. in parent_inkey column drop a row that contains pinkeylist

In [ ]:
def dropRowsbycolumnlist(df, cn, cl):
    '''
    This functions allows one to drop rows by column list:
        df = DF  ## Dataframe containing all information
        cn = colName ## Column name to check molecules to drop
        cl = mollist ## List of moleucles present in column to be dropped  
        
    e.g. I want to drop a list of molecules by parent_inkey:
        df = DF
        cn = 'parent_inkey'
        cl = ['INFDPOAKFNIJBF-UHFFFAOYSA-N', 'SYJFEGQWDCRVNX-UHFFFAOYSA-N',
              'CTSLUCNDVMMDHG-UHFFFAOYSA-N', 'ZFSLODLOARCGLH-UHFFFAOYSA-N']
       
    '''
    x = df[~df[cn].isin(cl)]
    print (len(x))
    return x  

## 2.5. Converting functions from smiles to 3D coordinates using RDKIT

In [ ]:
def addMolFromSmiletoPandasDF(df, smiColname, molColname):
    '''
        Function to add a molecule (in 1 Dimension) column to pandas dataframe from Smiles provided 
        
        Example:
        
        df = inditexdf ### pandasDataframe name
        smiColname = 'smiles' ## the smiles column name 
        molColname='mol1D'
        
        addMolFromSmiletoPandasDF(df, smiColname, molColname)
    '''
    
    return PandasTools.AddMoleculeColumnToFrame(df,  smilesCol=smiColname, molCol=molColname)

In [ ]:
def mol1Dto2Drdkit( x, smi = False ):
    '''
        Convert from smiles or 1D molecule to 2D using RDkit
        
        Example:
        
        If you got a pandas dataframe with 1D molecule (mol1D column) or smiles
        (smi column)run this function:
            - smile provided:
                df['mol2D'] = df.smi.apply(lambda x: mol1Dto2Drdkit(x))
            - mol provided:
                df['mol2D'] = df.mol1D.apply(lambda x: mol1Dto2Drdkit(x))
    '''
    ### checking if x is smiles or molecule
    
    if smi == True:
        try:
            m = Chem.MolFromSmiles(x)
        except:
            print('ERROR: no smile provided')
    else:
        m = x
    
    ### Computing 2D coordinates
    
    AllChem.Compute2DCoords(m)
    
    return m

In [ ]:
def mol2Dto3Drdkit(m, rmHs= True):
    '''
        Convert from 2D molecule to 3D using RDkit
        
        Example:
        
        If you got a pandas dataframe with 2D molecule (mol2D column) run this function:
        
            rmHs = False ## False if one wants to keep Hidrogens, 
                         ## True if you want to remove them. 
        
        e.g. df['mol3D'] = df.mol2D.apply(lambda x: mol2Dto3Drdkit(x, rmHs = True))
        
    '''
    m2 = Chem.AddHs(m)
    AllChem.EmbedMolecule(m2,AllChem.ETKDG())
    if rmHs == False:
        return m2
    if rmHs == True:
        m3 = Chem.RemoveHs(m2)
        return m3

## 2.6. Adding Parent smiles, inchi and inchikey to DF

In [ ]:
def addParentInfo(dfname, molcol):
    '''
       Calculating smiles, inchi and inchikey from molecule and adding information
       in pandas Dataframe. 
       
       input parameters:
       dfname = DF ## dataframe which contains the molecule
       molcol = 'parent_mol' ## molecule column
    '''
    df = dfname.copy()
    df['parent_smiles'] = df[molcol].apply(lambda x: addParentsmi(x))
    df['parent_std_inchi'] = df[molcol].apply(lambda x: addParentinchi(x, 'STD'))
    df['parent_nonstd_inchi'] = df[molcol].apply(lambda x: addParentinchi(x, 'nonSTD'))
    df['parent_std_inkey'] = df[molcol].apply(lambda x: addParentinkey(x, 'STD'))
    df['parent_nonstd_inkey'] = df[molcol].apply(lambda x: addParentinkey(x, 'nonSTD'))
    return df

In [ ]:
def addParentsmi(mol):
    
    psmiles = Chem.MolToSmiles(mol, isomericSmiles=True)
    
    return (psmiles)

In [ ]:
def addParentinchi(mol, cat):

    '''
       This function returns standard or non standard inchi.
       mol = 'mol3D' ## molecule column name
       cat = 'STD' ## 'STD' if one wants standard inchi where no tautomerism, stereoisomerism is counted.
                   ## 'nonSTD' if one wants non-standard inchi . 
    '''
    
    if cat == 'STD':
        pinchi = Chem.MolToInchi(mol)        
    elif cat == 'nonSTD':
        pinchi = Chem.MolToInchi(mol, options='/FixedH')
    
    return (pinchi)

In [ ]:
def addParentinkey(mol, cat):
    '''
       This function returns inchi key standard or non standard.
       mol = 'mol3D' ## molecule column name
       cat = 'STD' ## 'STD' if one wants standard inchi key where no tautomerism, stereoisomerism is counted.
                   ## 'nonSTD' if one wants non-standard inchi key. 
    '''
    
    if cat == 'STD':
        pinchi = Chem.MolToInchi(mol)
        pinkey = Chem.InchiToInchiKey(pinchi)        
    elif cat == 'nonSTD':
        pinchi = Chem.MolToInchi(mol, options='/FixedH')
        pinkey = Chem.InchiToInchiKey(pinchi)
    
    return (pinkey)

## 2.7. Calculating 3D coordinates with CORINA

In [ ]:
def convertTo3DwithCORINA(data, outname):
    
    ''' 
        Function to calculate 3D coordinates with CORINA 
        An SDFile should be provided to generate coordinates.
        Examples:
        
        data = 'cmr.sdf' ## path to SDF file 
        outname = 'cmr-3D.sdf' ## give an output name to generate the output-3D.sdf
        convertTo3DwithCORINA(data, outname)
    '''
    corina = "/opt/corina/corina3494/corina " ## this version requires license 
#     corina = "/opt/corina/corina24/corina " ## old version but permanent for Manolo
    os.system(corina+"-dwh -dori -ttracefile=corina.trc -it=sdf "+data+
              " -ot=sdf "+outname)

## 2.8. Loading SDF into Pandas Dataframe

In [ ]:
def LoadSDFintoDF(filename, idName='ID', molColName='ROMol', includeFingerprints=False,
                  isomericSmiles=False, smilesName=None, embedProps=False):
    '''
        Read file in SDF format and return as Pandas data frame. 
        If embedProps=True all properties also get embedded in Mol objects in the molecule column. 
        If molColName=None molecules would not be present in resulting DataFrame (only properties 
        would be read). 
        I took that function from PandasTools and modified it eliminating the sanitize option and
        adding removeHs = False, cause I do not want the molecule to be modified. 
    ''' 


    if isinstance(filename, string_types):
        if filename.lower()[-3:] == ".gz":
            import gzip 
            f = gzip.open(filename, "rb") 
        else: 
            f = open(filename, 'rb') 
            close = f.close 
    else:
        f = filename
        close = None  # don't close an open file that was passed in 
    records = [] 
    indices = [] 
    for i, mol in enumerate(Chem.ForwardSDMolSupplier(f,removeHs=False)):
        if mol is None: 
            continue 
        row = dict((k, mol.GetProp(k)) for k in mol.GetPropNames()) 
        if molColName is not None and not embedProps: 
            for prop in mol.GetPropNames(): 
                  mol.ClearProp(prop) 
        if mol.HasProp('_Name'): 
            row[idName] = mol.GetProp('_Name') 
        if smilesName is not None: 
            row[smilesName] = Chem.MolToSmiles(mol, isomericSmiles=isomericSmiles) 
        if molColName is not None and not includeFingerprints: 
            row[molColName] = mol 
        elif molColName is not None: 
            row[molColName] = PandasTools._MolPlusFingerprint(mol) 
        records.append(row) 
        indices.append(i) 
   
    if close is not None:
        close() 
    PandasTools.RenderImagesInAllDataFrames(images=True) 
    return pd.DataFrame(records, index=indices) 

In [ ]:
# removeHs=True

## 2.8. Protonating at x pH with MOKA

In [ ]:
def protonateWithMoka(data, pH, outname):
    
    ''' 
        Function to calculate protonation state of the molecule at x pH
        using MOKA program.
        An SDFile should be provided.
        
        Example:
        
        data = 'cmr-3D.sdf' ## path to SDF file 
        pH = '7.0' ## give a pH at which you want to protonate the molecules 
        outname = 'cmr-3D-prot7.sdf' ## give an output name to generate the output-protonated.sdf
        protonateWithMoka(data, pH, outname)
    '''
    
    blabber = "/opt/moka/blabber_sd "
    os.system(blabber+data+" -p "+pH+" -o "+outname)

## 2.9. Aggregating State information to DF ( positive , negative, neutral )

In [ ]:
def state( df, col ):
    '''
       Input parameters:
            df = df ## pandas dataframe after protonating with moka
            col = 'NETCHARGE' ## column containin netcharge after protonating with moka
            
       This function will allow you to create a new column in the pandas dataframe called state
       where NETCHARGE column is taken to categorize the molecule protonation state with the 
       following conditions:
       
             - POSITIVE if NETCHARGE is greater than 0.5
             - NEGATVE if NETCHARGE is less than -0.5
             - NEUTRAL if -0.5>= NETCHARGE <= 0.5
             
       It returns: a pandas dataframe including 'state column'
       
    '''
    
    df[col].fillna(0, inplace=True)
    
    for i, row in df.iterrows():
        if float(row[col]) > 0.5:
            df.loc[i,'state'] = 'positive'
        elif float(row[col]) < -0.5:
             df.loc[i,'state'] = 'negative'
        elif float(row[col]) >= -0.5 and float(row[col]) <= 0.5:
             df.loc[i,'state'] = 'neutral'
    
    print ( 'Total Positive molecules are ' +str (len(df[df['state'] =='positive'])))
    print ( 'Total Negative molecules are ' +str (len(df[df['state'] =='negative'])))
    print ( 'Total Neutral molecules are ' +str (len(df[df['state'] =='neutral'])))
    
    return df

In [ ]:
def protonationState(inSDF, minitest = True):
    
    '''
        This Function should be used after using protonateWithMoka() function, 
        or after using moka to protonate at some pH the Dataset. 
        As an input one should give:
        
            data = 'cmr-3D.sdf' ## path to SDF file
            minitest = True or False ## True if you one to build a minitest set
                                     ## False if you want not to build a minitest set
                                     
        Outputs:
           1- If one ran with minitestset = True 
                e.g. miniDF, protStateDF = protonationState(inSDF, minitest = True)
            
              The output will be two DF: 
                - miniDF:  containing whole minitestset molecules,
                - protStateDF: containing the whole molecules
           2- If minitest is set up to False then:
                e.g. protStateDF = protonationState(inSDF, minitest = False)
                
                The output will be one DF: 
                - protStateDF: containing the whole molecules
            
    '''
    
    ## Loading SDFile in pandas DF
    molcol = 'mol3Dprot'
    protDF = PandasTools.LoadSDF(inSDF, molColName=molcol)
    protDF.drop(['ID'], axis= 1, inplace=True)
    
    protDF['NETCHARGE'].fillna(0, inplace=True)
    print('Total Original DF are : ' +str(len(protDF)))
    protDF['NETCHARGE'] = protDF['NETCHARGE'].apply(pd.to_numeric)
    
    ## Positive section:
    posDF = protDF[protDF['NETCHARGE']>0.5]
    posDF.insert(len(posDF.columns),'State','positive') ## insterting a new column named State
    print ( 'Total positive molecules  are : ' + str(len(posDF)))
           
    ## Negative section:
    negDF = protDF[protDF['NETCHARGE']<-0.5]
    negDF.insert(len(negDF.columns),'State','negative') ## insterting a new column named State
    print ( 'Total negative molecules  are : ' + str(len(negDF)))
    
    ## Neutral section:
    neutralDF =  protDF[(protDF['NETCHARGE']>=-0.5) & (protDF['NETCHARGE']<= 0.5)]
    neutralDF.insert(len(neutralDF.columns),'State','neutral') ## insterting a new column named State
    print ( 'Total neutral molecules  are : ' + str(len(neutralDF)))
    
    ## Concatenating: 
    
    protStateDF = pd.concat([posDF, negDF, neutralDF])
    protStateDF =  protStateDF.reset_index(drop=True)
    print ( 'Total pos+neg+neutral molecules  are : ' + str(len(protStateDF)))
#     writeSDFfromPandasDF(protStateDF, 'PC-norm-3D-prot-state.sdf', 'mol3Dprot', list(protStateDF.columns))

    # Minitest section: (OPTIONAL)
    def intRound(dataframe):
        '''
        This little function is giving a random number to choose for a minitest set
        Needs to be improved
        '''
        x = len(dataframe)

        if x <= 10:
            mn = x
            return (mn)
        elif x > 10 and x <= 100 :
            mn = int(np.around((len(dataframe)/6 ), decimals=0)) 
            return (mn)
        elif x > 100 :
            mn = int(np.around((len(dataframe)/100) , decimals=0))
            return (mn)
    
    if minitest == True:
        print ( '\nBuilding Minitest set')                
             
        
        if len(posDF)>1:
            mininum = intRound(posDF)
            minipos = posDF.iloc[:mininum,:]
        
        if len(negDF)>1:
            mininum = intRound(negDF)
            minineg = negDF.iloc[:mininum,:]
        
        if len(neutralDF)>1:
            mininum = intRound(neutralDF)
            minineutral = neutralDF.iloc[:mininum,:]
        
        minitestsetDF = pd.concat([minipos, minineg, minineutral])
        minitestsetDF =  minitestsetDF.reset_index(drop=True)
        print ( 'Total minitest molecules  are : ' + str(len(minitestsetDF)))
#         ### creating minitest directory plus SDF inside:
        
#         directory = os.getcwd()+'/minitest'
        
#         if not os.path.exists(directory):
#             os.makedirs(directory)
#         writeSDFfromPandasDF(minitestsetDF, directory+'/minitestPC.sdf', 'mol3Dprot', list(minitestsetDF.columns))
        
        return (minitestsetDF, protStateDF)
    else:
        return (protStateDF)
    

## 2.9. Function to create directories

In [ ]:
def createDir(vpath, dirname):
    '''
    This functions checks if dirname exists in path given, 
    if does not exists, then it will be created. 
    
    e.g. 
    
        vpath = os.getcwd() ## current directory path
        dir2Dname = '2-2Dcoord' ### directory name to be created

        createDir(vpath, dir2Dname)
    '''
    directory = vpath+'/'+dirname

    if not os.path.exists(directory):
        os.makedirs(directory)
        print (dirname + ' is created')
    else: 
        print(dirname + ' already exists')

## 2.10. Writing SDFile from Pandas Data Frame

In [ ]:
def writeSDFfromPandasDF(df, output, molColName, props):
    '''
        Function that allows one writing an SDFile from a pandas dataframe with the properties
        that one want to keep it
        
        Example:
        
        df = inditexdf ### pandasDataframe name
        output = 'inditex3D.sdf' ## give an output SD file name
        molColName = 'mol2D'## give the column name where the molecule is stored
        props = ['smiles','cas', 'annotation'] ## list of columns names you want to add in SDFile as properties
        
        if one want to add all properties:
        props = list(df.columns)
        
        writeSDFfromPandasDF(df, output, molColName, props)
            
    '''
    
    PandasTools.WriteSDF(df, output, molColName=molColName, properties=props)
    

## 2.11. Normalization automatic protocol (ready to create model)

In [ ]:
def normRDkit2D(df, endpoint, molcol, vpath, pH, activity, method):
    '''
    This function converts a 1D molecule to a protonated 3D molecule, eliminating duplicates.
    
    1D --> 2D --> calculate parent inkey --> check duplicates by parent inkey --> Drop duplicates  --> 
    Curate duplicates info --> Save SDFile
    
    Input parameters:
    
        df = incDF
        endpoint= 'miniBCF'
        molcol= 'mol1D'
        vpath = os.getcwd() ## current directory path
        pH = '7'
        activity = 'logkb' ### activity field
        method = 'quantitative' ## set quantitative for numbers
                                ## set qualitative for negative/positive (0/1)

    This function returns a normalized pandas DF with prot and 3D structures without duplicates, 
    and the list of molecules that were duplicated
    
    e.g. normDF, duplist = normwithRDkit(df, endpoint, molcol, vpath, pH, activity, method)
    
    '''
    
    ## Creates 2D SDfile in the folder:
    
    print ('\033[1m' + '\n1.  Writing 2D SDFile'+'\033[0m')
    dir2Dname = '2-2Dcoord'
    createDir(vpath, dir2Dname)
    
    df['mol2D'] = df[molcol].apply(lambda x: mol1Dto2Drdkit(x))
    del df[molcol]
    
    ## Writing 2D SDFile
    out2Dfile = vpath+'/'+dir2Dname+'/'+endpoint+'-2D.sdf'
    props = list(df.columns)
    print ('Total 2D molecules : '+str(len(df)))
    writeSDFfromPandasDF(df, out2Dfile, 'mol2D', props)
    
    ## Calculating parent smiles, inchi and inchikey
    
    df = addParentInfo(df, 'mol2D')
       
    ## Checking duplicates
    print ('\033[1m' + '\n2.  Checking and Dropping duplicates'+'\033[0m') 
    
    duplist = getDupSingleDF(df, 'parent_inkey')
    
    ## Dropping duplicated molecules
    
    normDF = dropDupMols(df,'parent_inkey')
    
    ### After dropping molecules one can convert cell list with one string to string, and 
    ### float cells list to give the mean value:
    for i, x in normDF.iterrows():
        for j in normDF.columns:
            if type(x[j]) == list:
                if len(x[j]) == 1:
                    normDF[j][i]= x[j][0]
                elif j == 'mol2D':
                    normDF['mol2D'][i]= x['mol2D'][0]
                elif j == activity:
                    if method == 'quantitative':
                        normDF[activity][i]= np.mean(np.array(x[activity]).astype(np.float))
                    else:
                        normDF[activity][i]= np.min(x[activity]).astype(np.float) 
            else:
                continue


    ## Writing 2D SDfile
    props = list(normDF.columns)
    
    outnormfile =  vpath+'/'+dir2Dname+'/'+endpoint+'2D_norm.sdf'
    writeSDFfromPandasDF(normDF, outnormfile, 'mol2D', props)

    print ('Total 2D Normalized molecules : ' + str(len(PandasTools.LoadSDF(out2Dfile))))    
    
    ## Model Building preparation inputs parameters
   
    ## Model Building preparation inputs parameters
    
    dirModelname = '3-modelSDF' ## name directory to save 2D sdFile with 2D coordinates
    molcol = 'mol2D'
    name = 'parent_inkey'
    
    trainDF, testDF, globalDF = randomsplit(normDF, dirModelname, activity, molcol, name, endpoint, vpath)
    
    return (normDF, duplist)

In [ ]:
def normwithRDkit(df, endpoint, molcol, vpath, pH, activity, method):
    '''
    This function converts a 1D molecule to a protonated 3D molecule, eliminating duplicates.
    
    1D --> 2D --> protonate --> Add Hidrogens --> Calculate 3D Coord --> Remove Hidrogens -->
    calculate parent inkey --> check duplicates by parent inkey --> Drop duplicates  --> 
    Curate duplicates info --> Save SDFile
    
    Input parameters:
    
        df = incDF
        endpoint= 'miniBCF'
        molcol= 'mol1D'
        vpath = os.getcwd() ## current directory path
        pH = '7'
        activity = 'logkb' ### activity field
        method = 'quantitative' ## set quantitative for numbers
                                ## set qualitative for negative/positive (0/1)

    This function returns a normalized pandas DF with prot and 3D structures without duplicates, 
    and the list of molecules that were duplicated
    
    e.g. normDF, duplist = normwithRDkit(df, endpoint, molcol, vpath, pH, activity, method)
    
    '''
    
    ## Creates 2D SDfile in the folder:
    
    print ('\033[1m' + '\n1.  Writing 2D SDFile'+'\033[0m')
    dir2Dname = '2-2Dcoord'
    createDir(vpath, dir2Dname)
    
    df['mol2D'] = df[molcol].apply(lambda x: mol1Dto2Drdkit(x))
    del df[molcol]
    
    ## Writing 2D SDFile
    out2Dfile = vpath+'/'+dir2Dname+'/'+endpoint+'-2D.sdf'
    props = list(df.columns)
    print ('Total 2D molecules : '+str(len(df)))
    writeSDFfromPandasDF(df, out2Dfile, 'mol2D', props)
    
    ##### Protonate then 3D coordinates ######
       
    ## Creates protonation SDFile:
    print ('\033[1m' + '\n2.  Writing protonated SDFile at pH '+pH+'\033[0m')
    dirProtname = '3-protonated' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dirProtname)
    
    ## Writing protonated SDFile
    
    in2Dfile =  vpath+'/'+dir2Dname+'/'+endpoint+'-2D.sdf'
    outProtfile =  vpath+'/'+dirProtname+'/'+endpoint+'-prot.sdf'
    protonateWithMoka(in2Dfile, pH, outProtfile)
    print ('Total prot molecules : ' + str(len(PandasTools.LoadSDF(outProtfile))))

    ## Creates 3D SDfile in the folder:
    
    print ('\033[1m' + '\n3.  Writing Prot 3D SDFile'+'\033[0m')
    dir3Dname = '4-3D-prot' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dir3Dname)
    
    protDF = PandasTools.LoadSDF(outProtfile, molColName='molprot')
      
    del protDF['ID']
    protDF['mol3Dprot'] = protDF.molprot.apply(lambda x: mol2Dto3Drdkit(x,False))
    del protDF['molprot']
    
    protDF = addParentInfo(protDF, 'mol3Dprot')
    
    ## Adding protonation STATE column 
    
    print ('\033[1m' + '\n4.  Adding Protonation State info'+'\033[0m')    
    protDF = state( protDF, 'NETCHARGE' )
    
    ## Checking duplicates
    print ('\033[1m' + '\n5.  Checking and Dropping duplicates'+'\033[0m') 
    
    duplist = getDupSingleDF(protDF, 'parent_inkey')
    
    ## Dropping duplicated molecules
    
    normDF = dropDupMols(protDF,'parent_inkey')
    
    ### After dropping molecules one can convert cell list with one string to string, and 
    ### float cells list to give the mean value:
    for i, x in normDF.iterrows():
        for j in normDF.columns:
            if type(x[j]) == list:
                if len(x[j]) == 1:
                    normDF[j][i]= x[j][0]
                elif j == 'mol3Dprot':
                    normDF['mol3Dprot'][i]= x['mol3Dprot'][0]
                elif j == activity:
                    if method == 'quantitative':
                        normDF[activity][i]= np.mean(np.array(x[activity]).astype(np.float))
                    else:
                        normDF[activity][i]= np.min(x[activity]).astype(np.float) 
            else:
                continue


    ## Writing 3D SDfile
    props = list(normDF.columns)
    
    out3Dfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-prot-3D.sdf'
    writeSDFfromPandasDF(normDF, out3Dfile, 'mol3Dprot', props)

    print ('Total Prot 3D molecules : ' + str(len(PandasTools.LoadSDF(out3Dfile))))    
    
    ## Model Building preparation inputs parameters
   
    ## Model Building preparation inputs parameters
    
    dirModelname = '5-model' ## name directory to save 3D sdFile with 3D coordinates
    molcol = 'mol3Dprot'
    name = 'parent_inkey'
    
    trainDF, testDF, globalDF = randomsplit(normDF, dirModelname, activity, molcol, name, endpoint, vpath)
    
    return (normDF, duplist)

In [ ]:
def normwithCorina(df, endpoint, molcol, vpath, pH, activity, method):
    '''
    This function converts a 1D molecule to a protonated 3D molecule, eliminating duplicates.
    
    1D --> 2D --> protonate --> Add Hidrogens --> Calculate 3D Coord -->
    calculate parent inkey --> check duplicates by parent inkey --> Drop duplicates  --> 
    Curate duplicates info --> Save SDFile
    
    Input parameters:
    
        df = incDF
        endpoint= 'miniBCF'
        molcol= 'mol1D'
        vpath = os.getcwd() ## current directory path
        pH = '7'
        activity = 'logkb' ### activity field
        method = 'quantitative' ## set quantitative for numbers
                                ## set qualitative for negative/positive (0/1)

    This function returns a normalized pandas DF with prot and 3D structures without duplicates, 
    and the list of molecules that were duplicated
    
    e.g. normDF, duplist = normwithRDkit(df, endpoint, molcol, vpath, pH, activity, method)
    
    '''
    
    ## Creates 2D SDfile in the folder:
    
    print ('\033[1m' + '\n1.  Writing 2D SDFile'+'\033[0m')
    dir2Dname = '2-2Dcoord'
    createDir(vpath, dir2Dname)
    
    df['mol2D'] = df[molcol].apply(lambda x: mol1Dto2Drdkit(x))
    del df[molcol]
    
    ## Writing 2D SDFile
    out2Dfile = vpath+'/'+dir2Dname+'/'+endpoint+'-2D.sdf'
    props = list(df.columns)
    print ('Total 2D molecules : '+str(len(df)))
    writeSDFfromPandasDF(df, out2Dfile, 'mol2D', props)
    
    ##### Protonate then 3D coordinates ######
       
    ## Creates protonation SDFile:
    print ('\033[1m' + '\n2.  Writing protonated SDFile at pH '+pH+'\033[0m')
    dirProtname = '3-protonated' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dirProtname)
    
    ## Writing protonated SDFile
    
    in2Dfile =  vpath+'/'+dir2Dname+'/'+endpoint+'-2D.sdf'
    outProtfile =  vpath+'/'+dirProtname+'/'+endpoint+'-prot.sdf'
    protonateWithMoka(in2Dfile, pH, outProtfile)
    print ('Total prot molecules : ' + str(len(PandasTools.LoadSDF(outProtfile))))
   
    ########## Convert 3D structures with corina ##########    
    
    ## Creates 3D SDfile in the folder:
    
    print ('\033[1m' + '\n3.  Writing Prot 3D SDFile'+'\033[0m')
    dir3Dname = '4-3D-prot' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dir3Dname)

    ## Writing 3D SDfile
    in3Dfile = outProtfile
    out3Dfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-prot-3D.sdf'
    convertTo3DwithCORINA(in3Dfile,out3Dfile)
    print ('Total 3D molecules : ' + str(len(PandasTools.LoadSDF(out3Dfile))))
    
    protDF = LoadSDFintoDF(out3Dfile, molColName='mol3Dprot')
    del protDF['ID']
    
    protDF = addParentInfo(protDF, 'mol3Dprot')
    
    ## Adding protonation STATE column 
    
    print ('\033[1m' + '\n4.  Adding Protonation State info'+'\033[0m')    
    protDF = state( protDF, 'NETCHARGE' )
    
    ## Checking duplicates
    print ('\033[1m' + '\n5.  Checking and Dropping duplicates'+'\033[0m') 
    
    duplist = getDupSingleDF(protDF, 'parent_inkey')
    
    ## Dropping duplicated molecules
    
    normDF = dropDupMols(protDF,'parent_inkey')
    
    ### After dropping molecules one can convert cell list with one string to string, and 
    ### float cells list to give the mean value:
    for i, x in normDF.iterrows():
        for j in normDF.columns:
            if type(x[j]) == list:
                if len(x[j]) == 1:
                    normDF[j][i]= x[j][0]
                elif j == 'mol3Dprot':
                    normDF['mol3Dprot'][i]= x['mol3Dprot'][0]
                elif j == activity:
                    if method == 'quantitative':
                        normDF[activity][i]= np.mean(np.array(x[activity]).astype(np.float))
                    else:
                        normDF[activity][i]= np.min(x[activity]).astype(np.float) 
            else:
                continue


    ## Writing 3D SDfile
    props = list(normDF.columns)
    
    out3Dfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-prot-3D.sdf'
    writeSDFfromPandasDF(normDF, out3Dfile, 'mol3Dprot', props)

    print ('Total Prot 3D molecules : ' + str(len(PandasTools.LoadSDF(out3Dfile))))    
    
    ## Model Building preparation inputs parameters
    
    dirModelname = '5-model' ## name directory to save 3D sdFile with 3D coordinates
    molcol = 'mol3Dprot'
    name = 'parent_inkey'
    
    trainDF, testDF, globalDF = randomsplit(normDF, dirModelname, activity, molcol, name, endpoint, vpath)
    
    
    return (normDF, duplist)

In [ ]:
def CorinathenProt(df, endpoint, molcol, vpath, pH, activity, method):
    '''
    This function converts a 1D molecule to a protonated 3D molecule, eliminating duplicates.
    
    1D --> 2D --> protonate --> Add Hidrogens --> Calculate 3D Coord -->
    calculate parent inkey --> check duplicates by parent inkey --> Drop duplicates  --> 
    Curate duplicates info --> Save SDFile
    
    Input parameters:
    
        df = incDF
        endpoint= 'miniBCF'
        molcol= 'mol1D'
        vpath = os.getcwd() ## current directory path
        pH = '7'
        activity = 'logkb' ### activity field
        method = 'quantitative' ## set quantitative for numbers
                                ## set qualitative for negative/positive (0/1)

    This function returns a normalized pandas DF with prot and 3D structures without duplicates, 
    and the list of molecules that were duplicated
    
    e.g. normDF, duplist = normwithRDkit(df, endpoint, molcol, vpath, pH, activity, method)
    
    '''
    
    ## Creates 2D SDfile in the folder:
    
    print ('\033[1m' + '\n1.  Writing 2D SDFile'+'\033[0m')
    dir2Dname = '2-2Dcoord'
    createDir(vpath, dir2Dname)
    
    ### Calculating 2D coordinates
    
    df['mol2D'] = df[molcol].apply(lambda x: mol1Dto2Drdkit(x))
    del df[molcol]
    
    ############ Checking duplicates ###############
    
    print ('\033[1m' + '\n2.  Checking and Dropping duplicates'+'\033[0m') 
    df = addParentInfo(df, 'mol2D')
    
    duplist = getDupSingleDF(df, 'parent_inkey')
    
    ########## Dropping duplicated molecules #############
    
    normDF = dropDupMols(df,'parent_inkey')
    
    ### After dropping molecules one can convert cell list with one string to string, and 
    ### float cells list to give the mean value:
    for i, x in normDF.iterrows():
        for j in normDF.columns:
            if type(x[j]) == list:
                if len(x[j]) == 1:
                    normDF[j][i]= x[j][0]
                elif j == 'mol2D':
                    normDF['mol2D'][i]= x['mol2D'][0]
                elif j == activity:
                    if method == 'quantitative':
                        normDF[activity][i]= np.mean(np.array(x[activity]).astype(np.float))
                    else:
                        normDF[activity][i]= np.min(x[activity]).astype(np.float) 
            else:
                continue

                
    ## Writing 2D SDFile
    out2Dfile = vpath+'/'+dir2Dname+'/'+endpoint+'-2D.sdf'
    props = list(df.columns)
    print ('Total 2D molecules : '+str(len(df)))
    writeSDFfromPandasDF(df, out2Dfile, 'mol2D', props)
    
    
    ########## Convert 3D structures with corina ##########    
    
    ## Creates 3D SDfile in the folder:
    
    print ('\033[1m' + '\n3.  Writing 3D SDFile'+'\033[0m')
    dir3Dname = '3-3D' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dir3Dname)

    ## Writing 3D SDfile
    in3Dfile = out2Dfile
    out3Dfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-3D.sdf'
    convertTo3DwithCORINA(in3Dfile,out3Dfile)
    print ('Total 3D molecules : ' + str(len(PandasTools.LoadSDF(out3Dfile))))
    
    ########## Protonate with MOKA ##############
       
    ## Creates protonation SDFile:
    print ('\033[1m' + '\n4.  Writing protonated SDFile at pH '+pH+'\033[0m')
    dirProtname = '4-protonated' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dirProtname)
    
    ## Writing protonated SDFile
    
    inProtfile =  out3Dfile
    outProtfile =  vpath+'/'+dirProtname+'/'+endpoint+'-prot.sdf'
    protonateWithMoka(inProtfile, pH, outProtfile)
    print ('Total prot molecules : ' + str(len(PandasTools.LoadSDF(outProtfile))))
   
    
    protDF = LoadSDFintoDF(outProtfile, molColName='mol3Dprot')
    del protDF['ID']
    
    protDF = addParentInfo(protDF, 'mol3Dprot')
    
    ## Adding protonation STATE column 
    
    print ('\033[1m' + '\n5.  Adding Protonation State info'+'\033[0m')    
    protDF = state( protDF, 'NETCHARGE' )

    ## Writing 3D SDfile
    props = list(protDF.columns)
    
    out3DProtfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-prot-3D.sdf'
    writeSDFfromPandasDF(protDF, out3DProtfile, 'mol3Dprot', props)

    print ('Total 3D Prot molecules : ' + str(len(PandasTools.LoadSDF(out3DProtfile))))    
    
    ## Model Building preparation inputs parameters
    
    dirModelname = '5-model' ## name directory to save 3D sdFile with 3D coordinates
    molcol = 'mol3Dprot'
    name = 'parent_inkey'
    
    trainDF, testDF, globalDF = randomsplit(protDF, dirModelname, activity, molcol, name, endpoint, vpath)
    
    
    return (protDF, duplist)

## 2.11. Protonating
 - http://www.rdkit.org/docs/Cookbook.html --> Neutralizing Charged Molecules section

In [ ]:
""" contribution from Hans de Winter """
from rdkit import Chem
from rdkit.Chem import AllChem

def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None
def NeutraliseCharges(smiles, reactions=None):
    '''
        smis=["c1cccc[nH+]1",
          "C[N+](C)(C)C","c1ccccc1[NH3+]",
          "CC(=O)[O-]","c1ccccc1[O-]",
          "CCS",
          "C[N-]S(=O)(=O)C",
          "C[N-]C=C","C[N-]N=C",
          "c1ccc[n-]1",
          "CC[N-]C(=O)CC"]
        for smi in x:
            (molSmiles, neutralised) = NeutraliseCharges(smi)
            print(smi + "->" + molSmiles)    
    '''
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
    mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            mol = rms[0]
    if replaced:
        return (Chem.MolToSmiles(mol,True), True)
    else:
        return (smiles, False)

In [ ]:
# def _InitialiseNeutralisationReactions():
#     patts= (
#         # Imidazoles
#         ('[n+;H]','n'),
#         # Amines
#         ('[N+;!H0]','N'),
#         # Carboxylic acids and alcohols
#         ('[$([O-]);!$([O-][#7])]','O'),
#         # Thiols
#         ('[S-;X1]','S'),
#         # Sulfonamides
#         ('[$([N-;X2]S(=O)=O)]','N'),
#         # Enamines
#         ('[$([N-;X2][C,N]=C)]','N'),
#         # Tetrazoles
#         ('[n-]','[nH]'),
#         # Sulfoxides
#         ('[$([S-]=O)]','S'),
#         # Amides
#         ('[$([N-]C=O)]','N'),
#         )
#     return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]


# _reactions=None

# def NeutraliseCharges(smiles, reactions=None):
#     global _reactions
#     if reactions is None:
#         if _reactions is None:
#             _reactions=_InitialiseNeutralisationReactions()
#         reactions=_reactions
#     mol = Chem.MolFromSmiles(smiles)
#     replaced = False
#     for i,(reactant, product) in enumerate(reactions):
#         while mol.HasSubstructMatch(product):
#             replaced = True
#             rms = AllChem.ReplaceSubstructs(mol, product, reactant)
#             mol = rms[0]
#     if replaced:
#         return (Chem.MolToSmiles(mol,True), True)
#     else:
#         return (smiles, False)

    
# def _InitialiseNeutralisationReactions():
#     patts= (
#         # Imidazoles
#         ('n','[n+;H]'),
#         # Amines
#         ('N','[N+;!H0]'),
#         # Carboxylic acids and alcohols
#         ('O','[$([O-]);!$([O-][#7])]'),
#         # Thiols
#         ('S','[S-;X1]'),
#         # Sulfonamides
#         ('N','[$([N-;X2]S(=O)=O)]'),
#         # Enamines
#         ('N','[$([N-;X2][C,N]=C)]'),
#         # Tetrazoles
#         ('[nH]','[n-]'),
#         # Sulfoxides
#         ('S','[$([S-]=O)]'),
#         # Amides
#         ('N','[$([N-]C=O)]'),
#         )
#     return [Chem.MolFromSmarts(y),(Chem.MolFromSmiles(x, False)) for x,y in patts]
    
    
# def protonateRDkit(smiles):
#     '''
#     Examples of using it:

#         smis=("c1cccc[nH+]1",
#               "C[N+](C)(C)C","c1ccccc1[NH3+]",
#               "CC(=O)[O-]","c1ccccc1[O-]",
#               "CCS",
#               "C[N-]S(=O)(=O)C",
#               "C[N-]C=C","C[N-]N=C",
#               "c1ccc[n-]1",
#               "CC[N-]C(=O)CC")
#         for smi in smis:
#             (molSmiles, neutralised) = protonateRDkit(smi)
#             print(smi + "->" + molSmiles)
#     '''
# #     global _reactions
# #     if reactions is None:
# #         if _reactions is None:
# #             _reactions=_InitialiseNeutralisationReactions()
# #         reactions=_reactions

#     reactions =_InitialiseNeutralisationReactions()
#     mol = Chem.MolFromSmiles(smiles)
#     replaced = False
# #     print (reactions)
#     for i,(reactant, product) in enumerate(reactions):
# #         print (i, smiles, Chem.MolToSmiles(reactant), Chem.MolToSmarts(product))
#         if mol.HasSubstructMatch(reactant):
#             replaced = True
#             rms = AllChem.ReplaceSubstructs(mol, reactant, product)
#             mol = rms[0]
# #             print (i, smiles, Chem.MolToSmiles(mol))
#         else:
#             replaced = False
# #             mol = mol
#     if replaced:
#         return (Chem.MolToSmiles(mol,True), True)
#     else:
#         return (smiles, False)

# 3. Descriptors

## 3.1. Calculating Morgan FingerPrints

In [ ]:
def calc_fp_arr( mols, radius ):
    '''
    This function allows one to calculate morgan fingerprints from a list of molecules:
    
    Example:
        radius = 4 ## define morgan fingerprint radius 
        mols = df.mol3DProt ## dataframe column containing molecules 
        res = calc_fp_arr( mols , radius)
        
        ## Example to run a PCA
        pca = PCA( n_components = 3 )
        pca.fit( res )
        x = pca.transform( res )
        x = pd.DataFrame( x )
        x.columns = [ 'PC1', 'PC2', 'PC3' ]
        df = df.join(x)
        
    Attention:
        If this function doesn't work, try to update your conda environment
    '''
    fplist = []
    for mol in mols:
        arr = np.zeros( (1,) )
        fp = AllChem.GetMorganFingerprintAsBitVect( mol, int(radius))
        DataStructs.ConvertToNumpyArray( fp, arr )
        fplist.append( arr )
    return np.asarray( fplist )

## 3.2. Running  PCA using external Descriptors 

In [ ]:
def getPCAscores(descriptors, n_components):
    '''
        This functions returns as an output a PCA pandas Dataframe with the number
        of components calculated 
        
        * descriptors : 
            - numpy array with descriptors e.g. descriptors = [[0,0,1],[...],[1,0,1]]
            - or pandas dataframe columns  e.g. descriptors = df_piv.iloc[:,2:]
        * n_components: number of components 
    '''

    pca = PCA( n_components = n_components )
    pca.fit( descriptors )
    pcaDF = pca.transform( descriptors )
    pcaDF = pd.DataFrame( pcaDF )
    pcaDF.columns = [ 'PC'+str(i+1) for i in range(n_components) ]
    
    return pcaDF
    

# 4. Similarity Methods

## 4.1. Similarity between two Databases

In [ ]:
def getMorganfpDF(inSDF, radius):
   
    '''
        Getting morgan fingerprints from SDFile. The output will be a dataframe including
        in one column the morgan Fingerprints.
        
        inSDF  = 'ref.sdf'     ## Database SD file
        radius = 4             ## Define morgan fingerprint radius

    '''
    
    ### Model Analysis: Loading SDF or given pandasDF, calculating Morgan Fingerprints
    
    mfpDF = PandasTools.LoadSDF(inSDF, molColName='molRO') ## Dataframe with all Model information
    mfpDF['MorganFP']= [ AllChem.GetMorganFingerprintAsBitVect( mol, int(radius)) for mol in mfpDF.molRO]
    mfpDF['molID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(mfpDF))] ## adding ID column e.g. mol000001

    return (mfpDF)

def getSimilarity(mdDF,refDF, RFname, MDname, refID, mdID, cutoff, molField='molRO'):

    '''       
        Recommended protocol to be followed BEFORE running similarity analysis: 
        - Standardize molecules to eliminate contra ions, duplicates, molecules containing metal ions.... 
        - Double check manually if the molecules excluded are well excluded or they need to be included. 
        - Obtain 2D or 3D coordinates 
        - Protonate structures at the same pH (e.g. 7.4 pH) to be comparable
        - Calculate final smile, inchi and inchikey and 
        - name them as parent_smile, parent_inchi and parent_inchikey

        This function reads two pandas Dataframe where:
        - one contains a Reference Database
        - and the other one is the Model database
        
        Then a similiraty analysis using Morgan Fingerprints is performed at the cutoff provided

        INPUT Parameters to be defined:
            mdDF = modelDF         ## Model Dataframe including MorgaFP column previously calculated
            refDF = referenceDF    ## Reference Dataframe including MorgaFP column previously calculated
            RFname = 'Tox21'       ## Reference database basename to name files
            MDname = 'myModel'     ## Model database basename to name files
            refID = 'ToxCast_chid' ## Reference ID name column
            mdID = 'molID'         ## Model ID name column
            cutoff = 0.6           ## cutoff similarity distance by tanimoto Morgan fingerprints

        THREE OUTPUTs: same output in 3 different formats ( pandasDF, CSV and SDF)
                       The result are the molecules from the reference database
                       similars at the Model database ones at the CUTOFF provided.
                       The output will be sorted by:
                           - mdID       Ascending
                           - Similarity Descending
        
        e.g. Run first morgan finger prints calculation to get pandas DF from SDFiles:
        
        mdDF = getMorganfpDF(model.sdf, radius = 4)     ## Model Dataframe
        refDF = getMorganfpDF(reference.sdf, radis= 4)  ## Reference Dataframe 
        
        Then Run similarity analysis:
        
        simDF = getSimilarity(mdDF,refDF, RFname, MDname, refID, mdID, cutoff)
    '''    
     
    ## Loop to find similarity between two databases
    
    counter = 0
    simDF = pd.DataFrame([])
    
    refIDout = refID
    mdIDout = mdID
    if refID == mdID:
        refIDout += '_ref'
        mdIDout += '_model'

    for i, row_i in refDF.iterrows():       
        max_similarity = 0
        max_similarity_id = ''
        try:
            for j, row_j in mdDF.iterrows():
                sim = DataStructs.FingerprintSimilarity(row_j.MorganFP,row_i.MorganFP)
                if sim>max_similarity:
                    max_similarity = sim
                    max_similarity_mol = row_j[molField]
                    max_similarity_id = row_j[mdID]
        except:
            max_similarity = 0
            max_similarity_id= ''

        if max_similarity >= cutoff and max_similarity < 1:
            counter = counter+1
            simDict = [{refIDout:row_i[refID],mdIDout:max_similarity_id,
                        'similarity':float(max_similarity),RFname+'_mol': row_i[molField],
                        MDname+'_mol':max_similarity_mol}]
            tempDF = pd.DataFrame(simDict)
            tempDF = tempDF[[refIDout, mdIDout, 'similarity', RFname+'_mol',MDname+'_mol']]
            simDF = simDF.append(tempDF)
        else:
            pass

    print ('Total similar molecules found = ', counter)    
    
    if counter > 0:
        simDF = simDF.sort_values([mdIDout, 'similarity'], ascending=[True, False])
        simDF = simDF.reset_index(drop=True)
        simDF.to_csv('similarityAnalysis_'+RFname+'_vs_'+MDname+'_morganFP.csv',
                     sep='\t', encoding='utf-8', index=False )

        writeSDFfromPandasDF(simDF,'similarityAnalysis_'+RFname+'_vs_'+MDname+'_morganFP.sdf',
                             RFname+'_mol',list(simDF.columns))
    
    return (simDF)

## 4.2. Similarity by SMARTS

In [ ]:
def smartsFinding(DF,molDFname,ID, smarts):

    '''
        This functions allows one to perfom a searching by Substructure using SMARTS from a Dataframe
        
        DF = DF                ## Dataframe with molecules
        molDFname  = 'molCol'  ## molecule column 
        ID = 'name'            ## ID or name column
        smarts = '[NX3;H2,H1;!$(NC=O);!$(Nc)]' # aliphatic primary and secondary amines but no aniline nor amides
           or
        smarts = '[NX4;H0]'  # cuaternary amines (charged)
        
        smartsDF = smartsFinding(DF, molDFname,smarts)
        
        If one wants an SDFile, use writeSDFfromPandasDF(df, output, molColName, props) function
    '''

    target = Chem.MolFromSmarts(str(smarts))
    DF = DF.reset_index(drop=True)
    smartsDF = pd.DataFrame([]) ## Dataframe with all information DF plus matchDF
    matchDF = pd.DataFrame([]) ## Dataframe with matching information
    count = 0
   
    for i in range(len(DF)):
        mol = DF[molDFname][i]
        if mol.HasSubstructMatch(target):
            count += 1
            temp1 = pd.DataFrame(DF.loc[i,:]).T ## to convert row into Dataframe
            smartsDF = smartsDF.append(temp1)
            temp2 = {'HasSubMatch':'YES'}
            temp2 = pd.DataFrame([temp2])
            matchDF = matchDF.append(temp2)
        else:
            pass
#             temp1 = pd.DataFrame(DF.loc[i,:]).T ## to convert row into Dataframe
#             smartsDF = smartsDF.append(temp1)
#             temp2 = {'HasSubMatch':'NO'}
#             temp2 = pd.DataFrame([temp2])
#             matchDF = matchDF.append(temp2)
            

    print ('Total match molecules: ',count)
    smartsDF = smartsDF.reset_index(drop=True)
    matchDF = matchDF.reset_index(drop=True)
    smartsDF = smartsDF.join(matchDF)
#     smartsDF = smartsDF.sort_values(['HasSubMatch',ID], ascending=[False,True])
    
    return (smartsDF)

# 5. Applicability Domain

## 5.1. Histogram Applicability Domain analysis functions

In [ ]:
def scatterDC1vsDC2(vpath, RFD1, RFD2, MD1, MD2, RFname, MDname, DC1, DC2):

    '''
        This function generates an scatter plot of the descriptors used, e.g. MW vs LogP)
        vpath =              ## path to save files
        RFD1 = RFdf.LogP     ## ReF Database descriptors df column containing logp, PC1,... 
        RFD2 = RFdf.mw       ## ReF Database descriptors df column containing mw, PC2, ...
        MD1 = MDdf.LogP      ## Model Database descriptors df column containing logp, PC1, ...
        MD2 = MDdf.mw        ## Model Database descriptors df column containing mw, PC2, ...
        RFname = 'DrugBank'   ## ReF Database name
        MDname = 'Inditex'     ## Model Database name
        DC1 = 'PC1', 'LogP' ... ## name of descriptors or principal component used ...
        DC2 = 'PC2', 'MW' ...   ## name of descriptors or principal component used ...
    '''

    
    # Database information:
    xdb = RFD1  # logP from DrugBank Database
    ydb = RFD2    # MW from DrugBank Database

    ## Here we will calculate max() and min() for y and x to be added in the plots settings.

    xmax = xdb.max() + (xdb.max()/2) ### maximum of logP data
    xmin = xdb.min() + (xdb.min()/2) ### minimum of logP data
    ymax = ydb.max() + (ydb.max()/2) ### maximum of MW data
    ymin = ydb.min() + (ydb.min()/2) ### minimum of MW data
    
#     xmax = xdb.max() + 5 ### maximum of logP data
#     xmin = xdb.min() - 5 ### minimum of logP data
#     ymax = ydb.max() + 100 ### maximum of MW data
#     ymin = ydb.min() - 100 ### minimum of MW data


    # Model information:
    xmd = MD1  # logP from model Database
    ymd = MD2 # MW from model Database
    
    
    ######## Scatter Plot DC1 vs DC2 ##########

    my_dpi = 96 ### size of my figure

    ### 1st Plot: Scatter plot (model projected to drugbank) ###

    fig0 = plt.figure(figsize=(1250/my_dpi, 800/my_dpi))
    
    plt.title('Scatter Plot '+RFname+' vs '+MDname)
    plt.plot(xdb,ydb,'.k', color = '0.75') ### plot in grey
    plt.axis([xmin, xmax, ymin, ymax])

    # Plot data
    plt.plot(xmd,ymd,'.r')
    plt.axis([xmin, xmax, ymin, ymax])
    plt.xlabel(DC1)
    plt.ylabel(DC2)

    plt.show()
    fig0.savefig(vpath+"scatter_"+RFname+"_vs_"+MDname+"_"+DC1+"_"+DC2+".png")
    plt.close()
    mpl.rcParams.update(mpl.rcParamsDefault) ## set this to change to default matplotlib params
     
    return fig0

In [ ]:
def countPieChart(vpath, A, B, C, bins, RFname, MDname):

    '''
        This function generates a Counts pie chart of the applicability domain where:
            - A = Model + Reference compound found
            - B = Only Reference compound found
            - C = Only Model compound found 
        vpath = '1-histogramAnalysisResults'+str(bins)+'/'
        bins = 75        ## Same bins for model and ref database
        RFname = DrugBank ## ReF Database name
        MDname = Inditex ## Model Database name
    '''

    ############# countsPieChart ############

    # make a square figure and axes
    fig1 = figure(0, figsize=(6,6))
    ax = axes([0.1, 0.1, 0.8, 0.8])

    # The slices will be ordered and plotted counter-clockwise.
    labels = 'A', 'B', 'C' # A = DB & MD, B = DB, C = MD 
    fracs = [A, B, C]
    explode=(0.075, 0.075 , 0.075)
    colors= ["red", "lightblue", "yellow"]

    plt.pie(fracs, explode=explode, labels=labels, colors= colors,
                autopct='%1.1f%%', shadow=False, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.

    title('Counts Pie Chart using '+str(bins)+' bins')

    fig1.savefig(vpath+"Counts_piechart_"+RFname+"_vs_"+MDname+ "_" + str(bins) + "bins.png")
    plt.show()
    plt.close()
    mpl.rcParams.update(mpl.rcParamsDefault)

    return fig1

In [ ]:
def compoundPieChart(vpath, Ap, Bp, bins, RFname, MDname):

    '''
        This function generates a compounds pie chart of the applicability domain where:
            - A = Model + Reference compound found
            - B = Only Reference compound found
            - C = Only Model compound found 
        vpath = '1-histogramAnalysisResults'+str(bins)+'/'
        bins = 75        ## Same bins for model and ref database
        RFname = DrugBank ## ReF Database name
        MDname = Inditex ## Model Database name
    '''


    ########### Compounds Pie Chart ############

    # make a square figure and axes
    fig2 = figure(1, figsize=(6,6))
    ax = axes([0.1, 0.1, 0.8, 0.8])

    # The slices will be ordered and plotted counter-clockwise.
    labels = "A'", "B'" # A' = DB' & MD', B'= DB'
    fracs = [Ap, Bp]
    explode=(0.1, 0)
    colors= ["red", "lightblue"]

    pie(fracs, explode=explode, labels=labels, colors= colors,
                autopct='%1.1f%%', shadow=False, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.

    title('Compounds Pie Chart using '+str(bins)+' bins')
    
    fig2.savefig(vpath+"Compounds_piechart_"+RFname+"_vs_"+MDname+ "_" + str(bins) + "bins.png")
    plt.show()
    plt.close()
    mpl.rcParams.update(mpl.rcParamsDefault)
    
        
    return fig2

In [ ]:
def histogramPlot(vpath, xedges,yedges, Hmasked, xmdedges, ymdedges, Hmdmasked,
                  RFname, MDname, bins, DC1, DC2):

    '''
        This function generates a counts Histogram Plot of the applicability domain where:
            - A = Model + Reference compound found
            - B = Only Reference compound found
            - C = Only Model compound found
            - D = No Compound is found 
        
        xedges, yedges     ## x, y delimitation for Ref Database
        Hmasked            ## 2D coordinates for Ref Database
        xmdedges, ymdedges ## x, y delimitation for model Database
        Hmdmasked          ## 2D coordinates for model database
        vpath              ## path to save files
        bins             ## Same bins for model and ref database
        RFname = DrugBank ## ReF Database name
        MDname = Inditex ## Model Database name
        DC1 = 'PC1', 'LogP' ... ## name of descriptors or principal component used ...
        DC2 = 'PC2', 'MW' ...   ## name of descriptors or principal component used ...
    '''

    ########### Histogram Plot #############

    my_dpi = 96 ### size of my figure 

    ### Plot 2D Histogram  (model (in color scale) projected to drugbank (in grey scale)) ###

    # Plot 2D drugBank histogram using pcolor

    fig3 = plt.figure(figsize=(1250/my_dpi, 800/my_dpi))
   
    plt.title('Histogram Plot '+DC1+' vs '+DC2+' using '+str(bins)+' bins')
    plt.pcolormesh(xedges,yedges,Hmasked, cmap=mpl.cm.gray, vmin=-7, vmax=7) ### change vmin and vmax to put color scale

    # Plot 2D histogram using pcolor
    plt.pcolormesh(xmdedges,ymdedges,Hmdmasked)
    plt.xlabel(DC1)
    plt.ylabel(DC2)
       
    cmdbar = plt.colorbar()
    cmdbar.ax.set_ylabel('Counts ' + str(MDname) + ' Model')

    ####Saving plots in png format 
    
    fig3.savefig(vpath+"2DHist_"+RFname+"_vs_"+MDname+ "_"+str(bins)+"bins.png")
    plt.show()
    plt.close()
    mpl.rcParams.update(mpl.rcParamsDefault)
    
    return fig3

In [ ]:
def ratioCSVtable(vpath, RFname, MDname, bins, A,B,C,D,Ap,Bp,Cp,Dp):

    '''
        This function generates a Table in CSV format (tab separated) of the applicability domain where:
            For Counts:
            - A = Model + Reference compound found
            - B = Only Reference compound found
            - C = Only Model compound found
            - D = No Compound is found
            - A/B = ratio describing the percentage of counts covering the reference applicability domain 
            For Compounds:
            - Ap = Model + Reference compound found
            - Bp = Only Reference compound found
            - Cp = Only Model compound found
            - Dp = No Compound is found
            - Ap/Bp = ratio describing the percentage of compounds covering the reference applicability domain
        
        vpath = '1-histogramAnalysisResults'+str(bins)+'/' ## path to save files
        DBbins = 75        ## Same bins for model and ref database
        RFname = DrugBank ## ReF Database name
        MDname = Inditex ## Model Database name
    '''

    ###### Ratio A/B for counts and A'/B' for compounds ######
    
    ratiodict = [{'bins':bins,'AB':((A/B)*100), 'A':int(A), 
                  'B':int(B),'C':int(C), 'D':int(D), 'ApBp':((Ap/Bp)*100),
                  'Ap':int(Ap), 'Bp':int(Bp), 'Cp':int(Cp), 'Dp':int(Dp)}]   
    
    ratioDF = pd.DataFrame(ratiodict)

    ratioDF = ratioDF[['bins','AB', 'ApBp','A', 'B', 'C', 'D', 'Ap','Bp', 'Cp', 'Dp']]
    ratioDF.loc[:,('AB','ApBp')] = ratioDF[['AB','ApBp']].round(2)
    ratioDF.to_csv(vpath+'ratio_'+RFname+'_vs_'+MDname+ '_'+str(bins)+'bins.csv',
                   sep='\t', encoding='utf-8', index=False )
    
    return ratioDF

In [ ]:
def histAnalysis (RFD1, RFD2, MD1, MD2, bins, RFname, MDname, DC1, DC2):

    ''' 
        This is the main function that generates and analyses the data to check 
        the applicability domain. It calls the other functions to generating and
        saving plots, and tables.

        RFD1 = RFdf.LogP ## ReF Database descriptors df column containing logp, PC1,... 
        RFD2 = RFdf.mw   ## ReF Database descriptors df column containing mw, PC2, ...
        MD1 = MDdf.LogP ## ReF Database descriptors df column containing logp, PC1, ...
        MD2 = MDdf.mw   ## ReF Database descriptors df column containing mw, PC2, ...
        bins = 75        ## Same bins for model and ref database
        RFname = DrugBank ## ReF Database name
        MDname = Inditex ## Model Database name
        tune = 'YES' or 'NO' ## If someone wants to perform gridSizeOptimization put YES

        Pie Chart For Counts:
        - A = Model + Reference compound found
        - B = Only Reference compound found
        - C = Only Model compound found
        - D = No Compound is found
        - A/B = ratio describing the percentage of counts covering the reference applicability domain 
        Pie Chart For Compounds:
        - Ap = Model + Reference compound found
        - Bp = Only Reference compound found
        - Cp = Only Model compound found
        - Dp = No Compound is found
        - Ap/Bp = ratio describing the percentage of compounds covering the reference applicability domain
        

                                                                                                
                                           TWO DIFFERENT VIEWS                              
                                           -------------------                              
                    Table1. Number of counts                 Table2. Number of compounds    

                               REFERENCE                                 REFERENCE          
                       _________________________                 _________________________  
                      |            |            |               |            |            | 
                      |     YES    |     NO     |               |     YES    |     NO     | 
          ____________|____________|____________|   ____________|____________|____________| 
         |            |            |            |  |            |            |            | 
      M  |     YES    |      A     |      C     |  |     YES    |      Ap    |      Cp    | 
      O  |____________|____________|____________|  |____________|____________|____________| 
      D  |            |            |            |  |            |            |            | 
      E  |      NO    |      B     |      D     |  |      NO    |      Bp    |      Dp    | 
      L  |____________|____________|____________|  |____________|____________|____________| 

        
        
    '''
    
    ##### Defining path where to create a folder to save PNG images and tables ######
    vpath = 'histogramAnalysisResults-'+str(bins)+'/'
    
    if not os.path.exists(vpath):
        os.makedirs(vpath)
    else:
        shutil.rmtree(vpath,ignore_errors=True)
        os.mkdir(vpath)
       
    # Database information:
    
    xdb = RFD1  # logP from DrugBank Database
    ydb = RFD2    # MW from DrugBank Database
    DBbins = bins

    ## Here we will calculate max() and min() for y and x to be added in the plots settings. 
    
    xmax = xdb.max() + 5 ### maximum of logP data
    xmin = xdb.min() - 5 ### minimum of logP data
    ymax = ydb.max() + 100 ### maximum of MW data
    ymin = ydb.min() - 100 ### minimum of MW data
    
    # Model information:
    xmd = MD1  # logP from model Database
    ymd = MD2 # MW from model Database
    
    ###### Estimate the 2D histogram for Reference DATABASE ######
    
    xedges, yedges = np.linspace( xmin, xmax, DBbins ), np.linspace( ymin, ymax, DBbins ) ##### here you can define y and x delimitation and number of bins
    Href, xedges, yedges = np.histogram2d(xdb, ydb, (xedges, yedges))
    
    # Href needs to be rotated and flipped
    Href = np.rot90(Href)
    Href = np.flipud(Href)
    
    Hmasked = np.ma.masked_where(Href==0,Href) # Mask pixels with a value of zero

    ###### Estimate the 2D histogram for the MODEL database chosen ######

    MDbins = bins #### you can modify ModelBins if you want to amplify or reduce the spacing (size of the squares)

    xmdedges, ymdedges = np.linspace(xmin, xmax, MDbins), np.linspace(ymin, ymax, MDbins) ##### here you can define y and x delimitation and number of bins
    Hmd, xmdedges, ymdedges = np.histogram2d(xmd, ymd, (xmdedges, ymdedges))
 
    # Hmd needs to be rotated and flipped
    Hmd = np.rot90(Hmd)
    Hmd = np.flipud(Hmd)

    Hmdmasked = np.ma.masked_where(Hmd==0,Hmd) # Mask pixels with a value of zero

    ##########################################################################################
    #                                                                                        #
    #                                       TWO DIFFERENT VIEWS                              #
    #                                       -------------------                              #
    #                Table1. Number of counts                 Table2. Number of compounds    #
    #                                                                                        #
    #                           REFERENCE                                 REFERENCE          #
    #                   _________________________                 _________________________  #
    #                  |            |            |               |            |            | #
    #                  |     YES    |     NO     |               |     YES    |     NO     | #
    #      ____________|____________|____________|   ____________|____________|____________| #
    #     |            |            |            |  |            |            |            | #
    #  M  |     YES    |      A     |      C     |  |     YES    |      Ap    |      Cp    | #
    #  O  |____________|____________|____________|  |____________|____________|____________| #
    #  D  |            |            |            |  |            |            |            | #
    #  E  |      NO    |      B     |      D     |  |      NO    |      Bp    |      Dp    | #
    #  L  |____________|____________|____________|  |____________|____________|____________| #
    #                                                                                        #
    ##########################################################################################
        
    N = 4 ### we have 4 conditions
    nofCounts = np.zeros( N ) ### number of counts in 4 conditions from table 1
    nofCompounds = np.zeros( N )
#     fcp = open(vpath+"INFO_RefD_vs_"+MDname+"_compounds_"+str(DBbins)+".csv","w")
#     fcp.write('Reference_Database\tModel\tInfo\n')
    
    ### Href ad Hmd is an array contaning x(e.g. logP) and y(e.g. MW) coordinates 
    ### of the points to be histogrammed from the Reference database and 
    ### model respectively
    
    for i in range(DBbins-1):
        for j in range(DBbins-1):
            if (Href[i][j] > 0) and (Hmd[i][j] > 0):         # A and Ap) YES RFD YES Model 
                nofCounts[0] += 1
                nofCompounds[0] += Href[i][j]+ Hmd[i][j]
#                 fcp.write(str(Href[i][j])+"\t"+ str(Hmd[i][j])+ '\tYES RFD | YES MD\n')
            elif (Href[i][j] > 0) and (Hmd[i][j] == 0):       # B and Bp) YES RFD NO Model
                nofCounts[1] += 1
                nofCompounds[1] += Href[i][j]
#                 fcp.write(str(Href[i][j])+"\t"+ str(Hmd[i][j])+ '\tYES RFD | NO MD\n')
            elif (Href[i][j] == 0) and (Hmd[i][j] > 0):       # C and Cp) NO RFD YES Model
                nofCounts[2] += 1
                nofCompounds[2] += Hmd[i][j]
#                 fcp.write(str(Href[i][j])+"\t"+ str(Hmd[i][j])+ '\tNO RFD | YES MD\n')
            elif (Hmd[i][j] == 0) and (Href[i][j] == 0):       # D and Dp) NO RFD NO Model
                nofCounts[3] += 1
                nofCompounds[3] = 0
#                 fcp.write(str(Href[i][j])+"\t"+ str(Hmd[i][j])+ '\tNO RFD | NO MD\n')
    
#     fcp.close()

    A = nofCounts[0]
    B = nofCounts[1]
    C = nofCounts[2]
    D = nofCounts[3]
    Ap = nofCompounds[0]
    Bp = nofCompounds[1]
    Cp = nofCompounds[2]
    Dp = nofCompounds[3]
    
    ### Creating Plots plus text info ####
    
    ratioDF = ratioCSVtable(vpath, RFname, MDname, DBbins, A,B,C,D,Ap,Bp,Cp,Dp)
    scatterDC1vsDC2(vpath, RFD1, RFD2, MD1, MD2, RFname, MDname, DC1,DC2)    
    histogramPlot(vpath, xedges,yedges, Hmasked, xmdedges, ymdedges, Hmdmasked,
                  RFname, MDname, DBbins, DC1,DC2)
    countPieChart(vpath, A, B, C, DBbins, RFname, MDname)
    compoundPieChart(vpath, Ap, Bp, DBbins, RFname, MDname)
    
    AB = (A/B)*100
    ABp = (Ap/Bp)*100
  
    return ratioDF

        
        

In [ ]:
def gridSizeOptimization(RFD1, RFD2, MD1, MD2, RFname, MDname, DC1, DC2, start, stop, step):

    ''' 

        PART 4.1: Grid size optimization analysis:

        The optimum size of the grid ( number of divisions of the e.g. MW and e.g.logP axes)
        will be selected after analysing the results obtained with vaules ranging
        from 4 to 300 (one can change this as necessary) and identifying which will
        be the vaules too low or too high, leading to extreme and poorly informative
        results. 

        RFD1 = RFdf.LogP ## ReF Database descriptors df column containing logp, PC1,... 
        RFD2 = RFdf.mw   ## ReF Database descriptors df column containing mw, PC2, ...
        MD1 = MDdf.LogP ## ReF Database descriptors df column containing logp, PC1, ...
        MD2 = MDdf.mw   ## ReF Database descriptors df column containing mw, PC2, ...

        RFname = DrugBank ## ReF Database name
        MDname = Inditex ## Model Database name
        start = 4  ## beginning number 4
        stop = 300 ## final number 300
        step = 4   ## timestep from 4 to 4 (e.g. 4 8 12 16 ...)
    
    '''       
 
    os.system('mkdir gridSizeOptimization')
    
    newDF = pd.DataFrame([])
    
    for i in np.arange(start, stop, step):
        bins = i
        ratioOptDF = histAnalysis(RFD1, RFD2, MD1, MD2, bins, RFname,MDname, DC1,DC2)
        newDF = newDF.append(ratioOptDF)
    
    newDF.to_csv('gridSizeOptimization/ratio_'+RFname+'_vs_'+MDname+'_from'+str(start)+'to_'+str(stop)+'bins.csv',
                 sep='\t', encoding='utf-8', index=False )
    os.system('mv histogramAnalysisResults-* gridSizeOptimization/')
    
    newDF = newDF.reset_index(drop=True)

    return newDF

# 6.Bokeh interactive graphs

In [ ]:
from bokeh.plotting import figure, output_notebook, show, output_file, save
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool, WheelZoomTool
from bokeh.models import ColorBar, ResetTool, PanTool, LinearColorMapper, SaveTool

import bokeh.palettes as palettes
from bokeh.models.markers import *
# from bokeh.io import export_png
output_notebook()

In [ ]:
def scatter_mol(df,activity, DC1, DC2, molcol, catcol, name, img):
    """
    Makes an interactive scatter plot with 
    - df = pcaDF        ## dataframe contains molecules, principal components or descriptors, name of the molecules
    - activity = 'LogP' ## name of the activity field to color them 
    - mol = 'mol'       ## molecule column 
    - name = 'name'     ## name column 
    - DC1 = 'PC1'       ## column name first descriptor e.g. PC1, LogP, ...
    - DC2 = 'PC2'       ## column name first descriptor e.g. PC2, MW, ...
    - img = 'INCimg/'   ## folder path
    - catcol = 'State'  ## category column to color by them
    """
    
    xlabel = DC1
    ylabel = DC2
    def get_structures(mol_df, molcol, name, imgdir):
        img_path = []
        img_dir = imgdir
        
        if not os.path.exists(img_dir):  #  Checks if folder exists then creates it
            os.makedirs(img_dir)
            
        for i in range(len(mol_df)):
            Draw.MolToFile(mol_df[molcol][i],
                            img_dir + mol_df[name][i] + ".svg",
                            imageType="svg",
                            fitImage=False,
                            size=(200, 200))
            img_path.append(img_dir + mol_df[name][i] + ".svg")
        mol_df["img_path"] = img_path
        return mol_df

    bokehDF = get_structures(df, molcol, name, img)
    bokehDF = bokehDF.drop(molcol, axis = 1)    

    ## Choosing categories
    try:
        categories = np.unique(bokehDF[catcol]) ## creating categories by column
    except:
        bokehDF[catcol] = 'None'
        categories = np.array(['None']) ## creating categories by column
        
    
    ## Choosing colors
    if len(categories) < 5:
        mycolorlist = ['#d62728','#2ca02c','#6a3d9a','#c7c7c7'] ## red, green, violet, grey
    elif len(categories) >= 5 and len(categories) <= 10 :    
        mycolorlist = ['#ffff33', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                       '#000000', '#e377c2', '#7f7f7f', '#6a3d9a', '#17becf']
    
    catcolors = np.random.choice((len(mycolorlist)), len(categories),replace=False)
    pal = []
    for i in catcolors:
        pal.append(mycolorlist[i])
        
    ## Creating color dictionary by category
    colordict = dict(zip(categories, pal)) ## dictionary with category and color randomly asigned
    bokehDF["colors"] = [colordict[x] for x in bokehDF[catcol]]

    source = ColumnDataSource(bokehDF)
    hover = HoverTool(tooltips = 
                f"""
                <div>
                    <img src="@img_path" width="170" height="170"></img>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">@{name}</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">PCA:</span>
                    <span style="font-size: 12px;">@{DC1}, @{DC2}</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">Activity:</span>
                    <span style="font-size: 12px;">@{activity}</span>
                </div>

                """)

    TOOLS=[hover, CrosshairTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool()]
    p = figure(plot_width=800, plot_height=800, tools=TOOLS, x_axis_label=xlabel, y_axis_label=ylabel)
    p.scatter(x=DC1,y=DC2, size=5, alpha = 0.9, color = 'colors', legend=catcol, source=source)
    p.legend.location = "top_left"
    p.legend.click_policy="mute"
    show(p)
    return bokehDF

In [ ]:
def scatter_mol_RefvsMD(refDF,refDC1, refDC2, refmolcol, refName, refACT,refimg,
                        mdDF,mdDC1, mdDC2, mdmolcol, mdName, mdACT,mdimg):
    """
    Makes an interactive scatter plot with 
    
    Reference Dataframe parameters:
    
    - refDF = pcaDF      ## dataframe contains molecules, principal components or descriptors, name of the molecules
    - refDC1 = 'PC1'     ## column name first descriptor e.g. PC1, LogP, ...
    - refDC2 = 'PC2'     ## column name second descriptor e.g. PC1, LogP, ...
    - refmolcol = 'mol'  ## molecule column 
    - refName = 'name'   ## name column
    - refACT = 'LogP'    ## name of the activity field
    - refimg = 'REFimg/' ## Reference images folder path

    Model Dataframe parameters:
    
    - mdDF = pcaDF      ## dataframe contains molecules, principal components or descriptors, name of the molecules
    - mdDC1 = 'PC1'     ## column name first descriptor e.g. PC1, LogP, ...
    - mdDC2 = 'PC2'     ## column name second descriptor e.g. PC1, LogP, ...
    - mdmolcol = 'mol'  ## molecule column 
    - mdName = 'name'   ## name column
    - mdACT = 'LogP'    ## name of the activity field 
    - mdimg = 'MDimg/'  ## Model images folder path
    
   e.g. : Before running this one needs to assure both dataframe contains same columns names. 
   scatter_mol_RefvsMD(refDF, refDC1, refDC2, refmolcol, refName, refACT, 
                        refimg, mdDF, mdDC1, mdDC2, mdmolcol, mdName, mdACT, mdimg)
    """

    def get_structures(mol_df, molcol, name, imgdir):
        img_path = []
        img_dir = imgdir
        
        if not os.path.exists(img_dir):  #  Checks if folder exists then creates it
            os.makedirs(img_dir)
            
        for i in range(len(mol_df)):
            Draw.MolToFile(mol_df[molcol][i],
                            img_dir + mol_df[name][i] + ".svg",
                            imageType="svg",
                            fitImage=False,
                            size=(200, 200))
            img_path.append(img_dir + mol_df[name][i] + ".svg")
        mol_df.loc[:,'img_path'] = img_path ## labeling model
#         mol_df['img_path'] = img_path
        return mol_df


    ######## Reference Dataframe ########
    
#     refDF['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(refDF))]
#     refName = 'phiID'
    refDF = refDF[[refDC1, refDC2, refmolcol, refName, refACT]]
    refDF.columns = [refDC1, refDC2, 'mol', 'name', 'activity']
    bokeh_refDF = get_structures(refDF, 'mol', 'name', refimg)
#     bokeh_refDF['origin'] = 'ref' ## labeling reference
    bokeh_refDF.loc[:,'origin'] = 'ref' ## labeling model
    bokeh_refDF = bokeh_refDF.drop('mol', axis = 1)
    
    ######## Model Dataframe ########
#     mdDF['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(mdDF))]
#     mdName = 'phiID'
   
    mdDF = mdDF[[mdDC1, mdDC2, mdmolcol, mdName, mdACT]]
    mdDF.columns = [mdDC1, mdDC2, 'mol', 'name', 'activity']
    bokeh_mdDF = get_structures(mdDF, 'mol', 'name', mdimg)
    bokeh_mdDF.loc[:,'origin'] = 'model' ## labeling model
#     bokeh_mdDF['origin'] = 'model' ## labeling model
    bokeh_mdDF = bokeh_mdDF.drop('mol', axis = 1)
    
    ######### Concatenating reference and model Dataframes ########
    bokehDF = pd.concat([bokeh_refDF,bokeh_mdDF])

    
    ### colormap ###
    colormap = {'ref': '#cccccc', 'model': '#d9534f'}
    colors = [colormap[x] for x in bokehDF['origin']]
    bokehDF.loc[:,'colors'] = colors
#     bokehDF['colors'] = colors
    
    ### sizemap ###
    sizemap = {'ref': 10, 'model': 4}
    sizes = [sizemap[x] for x in bokehDF['origin']]
    bokehDF.loc[:,'sizes'] = sizes
    
#     ### markersmap ###
#     markersmap = {'ref': "x", 'model': "x"}
#     markers = [markersmap[x] for x in bokehDF['origin']]
#     bokehDF.loc[:,'markers'] = markers


#     bokehDF[refDC1] = pd.to_numeric(bokehDF[refDC1])
#     bokehDF[refDC2] = pd.to_numeric(bokehDF[refDC2])

    source = ColumnDataSource(bokehDF)
    hover = HoverTool(tooltips = 
                f"""
                <div>
                    <img src="@img_path" width="170" height="170"></img>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">@name</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">PCA:</span>
                    <span style="font-size: 12px;">@{refDC1}, @{refDC2}</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">Info:</span>
                    <span style="font-size: 12px;">@activity</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">Source:</span>
                    <span style="font-size: 12px;">@origin</span>
                </div>

                """)


    TOOLS=[hover, CrosshairTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool()]
    p = figure(plot_width=800, plot_height=800,tools=TOOLS)
    p.scatter(x=refDC1,y=refDC2, size='sizes',alpha=0.9, color='colors',legend='origin',source=source)
    p.legend.location = "top_left"
    p.legend.click_policy="mute"

    show(p)

# 7. etoxLAB

## 7.1. creating automatically a new model 

In [ ]:
def creatingNewModel(endpoint, descriptor, tag):
    '''
    This functions allows one to use etoxlab and create a new model automatically
        input:
            endpoint = 'ratDeG' ## endpoint name
            descriptor = 'adriana' ## descriptor name
            tag = 'degeneration' ## tag information
        
        e.g. creatingNewModel('ratDeG', 'adriana','degeneration')
    
    
    
    '''
    
    os.system('~/soft/eTOXlab/src/manage.py --new -e '+endpoint+' -t /toxicity/'+descriptor+'/'+tag+'/1')

    print ('Tag is set up')

    os.system('~/soft/eTOXlab/src/manage.py -v 0 -e '+endpoint+' --get=model')
    os.system('mv imodel.py '+descriptor+'_imodel.py')

## 7.2. RandomSplit

In [ ]:
def randomsplit(df, dirModelname, activity, molcol, name, endpoint,category, vpath):
    '''
        df = df                   ## pandas dataframe
        dirModelname = '5-model'  ## folder name to be created to store model files
        activity = 'activity'     ## columns with activity values
        molcol = 'mol3Dprot'      ## molecule column
        name = 'parent_inkey'     ## column name
        endpoint = 'BioTF'        ## endpoint name for basename files.sdf
        category = 'activity'     ## column name to create categories
        vpath = os.getcwd()       ## current directory path
        
        This functions returns 3 pandas dataframes, test, train and global DF
        The split is 70/30 train/test, with a randomseed = 1987
        It creates 3 SDFiles, global, test and train with files name, activity inside, 
        ready to build a model. 
        
        e.g. trainDF, testDF, globalDF = (df, dirModelname, activity, molcol, name, endpoint, category, vpath)
    '''
    

    
    ## Model Building preparation inputs parameters
   
    print ('\033[1m' + '\n Writing Model SDFile with less fields and Randomsplit'+'\033[0m')
    dirModelname = dirModelname ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dirModelname)
     
    ## Writing model Global SDfile
    globaldf = df.copy()
    globaldf = globaldf[[name, activity, molcol]]
    globaldf.columns = ['name', 'activity', 'mol']
    props = list(globaldf.columns)
    
    outModelfile =  vpath+'/'+dirModelname+'/global-'+endpoint+'.sdf'
    writeSDFfromPandasDF(globaldf, outModelfile, 'mol', props)
    
    ## Writing model training and test set SDfile:

    valdf = df[[name, activity, molcol]]
    valdf.columns = ['name', 'activity', 'mol']
    
    train, test, y_train, y_test = train_test_split(valdf, df[category], test_size=0.2, 
                                                    random_state= 1987, stratify=df[category])
    
    train
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    trainfile = vpath+'/'+dirModelname+'/tr-'+endpoint+'.sdf'
    testfile = vpath+'/'+dirModelname+'/pr-'+endpoint+'.sdf'
    
    trainprops = list(train.columns)
    testprops = list(test.columns)
    
    writeSDFfromPandasDF(train, trainfile, 'mol', trainprops)
    writeSDFfromPandasDF(test, testfile, 'mol', testprops)
    print ('Test : ' + str(len(test)))
    print ('Train : ' + str(len(train)))
    print ('Global : ' + str(len(df)))
    
    return ( train, test, df)

In [ ]:
def randomsplit_quantitative(df, dirModelname, activity, molcol, name, endpoint, catcol, vpath):
    '''
        df = df                   ## pandas dataframe
        dirModelname = '5-model'  ## folder name to be created to store model files
        molcol = 'mol3Dprot'      ## molecule column
        name = 'parent_inkey'     ## column name
        endpoint = 'BioTF'        ## endpoint name for basename files.sdf 
        vpath = os.getcwd()       ## current directory path
        
        This functions returns 3 pandas dataframes, test, train and global DF
        The split is 70/30 train/test, with a randomseed = 1987
        It creates 3 SDFiles, global, test and train with files name, activity inside, 
        ready to build a model. 
        
        e.g. trainDF, testDF, globalDF = (df, dirModelname, activity, molcol, name, endpoint, vpath)
    '''
    df = df.copy()
#     df = df[[name, activity, molcol]]
#     df.columns = ['name', 'activity', 'mol']
    
    ## Model Building preparation inputs parameters
   
    print ('\033[1m' + '\n Writing Model SDFile with less fields and Randomsplit'+'\033[0m')
    dirModelname = dirModelname ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dirModelname)
     
    ## Writing model Global SDfile
    props = list(df.columns)
    
    outModelfile =  vpath+'/'+dirModelname+'/global-'+endpoint+'.sdf'
    writeSDFfromPandasDF(df, outModelfile, molcol, props)
    
    ## Writing model training and test set SDfile:
    train, test, y_train, y_test = train_test_split(df,df[activity], test_size=0.2, 
                                                    random_state= 1987, stratify=df[catcol])
    
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    trainfile = vpath+'/'+dirModelname+'/tr-'+endpoint+'.sdf'
    testfile = vpath+'/'+dirModelname+'/pr-'+endpoint+'.sdf'
    
    trainprops = list(train.columns)
    testprops = list(test.columns)
    
    writeSDFfromPandasDF(train, trainfile, molcol, trainprops)
    writeSDFfromPandasDF(test, testfile, molcol, testprops)
    print ('Test : ' + str(len(test)))
    print ('Train : ' + str(len(train)))
    print ('Global : ' + str(len(df)))
    
    return ( train, test, df)

In [ ]:
def splitSDF (sdf, prop, seed):
    
    '''
        This function allows one to obtain a training and test set by
        randomspliting. 
        
        Input paremeters:
            sdf = file.sdf ### sd File input pathway
            prop = 70      ### give the percentage of splitting ( 70% trainset, 30% testset)
            seed = 483     ### give a random number 
        
        
        
        e.g. randomSplitSDF -f file.sdf -p 70 [-s 2356]
    '''
    prop = float(prop)
    seed = float(seed)

    nmols = 0
    try:
        f = open (sdf,'r')
    except:
        print ('unable to open file ',sdf, ' ABORT')
        exit(1)
        
    for line in f:
        if line.startswith('$$$$'):
            nmols = nmols+1
    f.close()

    ntrai = int(np.round(prop*nmols/100.0))
    npred = nmols - ntrai
    
    print (nmols, "compounds found. Creating series of ", ntrai, " for training and ", npred, " for prediction")

    if seed != None :
        npseed = int(seed)
        np.random.seed(npseed)
        
    elements = np.random.choice(nmols, ntrai, False)
    #print elements
    
    f  = open (sdf,'r')
    sdf = sdf.split("/")
    sdf = sdf[-1]
    fp = open ('pr-'+sdf,'w')
    ft = open ('tr-'+sdf,'w')

    i = 0
    for line in f:
        if i in elements :
            ft.write(line)
        else:
            fp.write(line)
        if line.startswith('$$$$'):
            i=i+1

    f.close()
    fp.close()
    ft.close()

# 8. Inditex

In [ ]:
def curationCMRDuplicates(df, colname):
    duplist = getDupSingleDF(df, colname)
    normDF = dropDupMols(df,colname)
    
    ### After dropping molecules one can convert cell list with one string to string, and 
    ### float cells list to give the mean value:
    for i, x in normDF.iterrows():
        for j in normDF.columns:
            if type(x[j]) == list:
                if len(x[j]) == 1:
                    normDF[j][i] = x[j][0]
                elif j == 'parent_mol':
                    normDF[j][i] = x[j][0]  
                elif j == 'Activity':
                    normDF['Activity'][i] = np.min(np.array(x['Activity']).astype(np.int))
                elif j == 'anntype':
                    if 'Negative' in normDF['anntype'][i]:
                        normDF['anntype'][i] = 'Negative'
            else:
                continue
                
    checkdupDF = normDF[normDF[colname].isin(duplist)]
    
    print('Original Dataset = '+ str(len(df)))
    print('Total number of duplicates = '+ str(len(duplist)))
    print('Normalized Dataset with no duplicates = '+ str(len(normDF)))
    print ('CMR compounds with Negative  = '+ str(len(normDF[normDF['anntype']=='Negative'])))
    print ('CMR compounds with Confirmed  = '+ str(len(normDF[normDF['anntype']=='Confirmed'])))
    print('Final Normalized + Duplicates =' + str(len(normDF)+len(duplist)))
    
    return (normDF, checkdupDF)

In [ ]:
def curationInditexDuplicates(df, colname):
    duplist = getDupSingleDF(df, colname)
    normDF = dropDupMols(df,colname)
    
    ### After dropping molecules one can convert cell list with one string to string, and 
    ### float cells list to give the mean value:
    for i, x in normDF.iterrows():
        for j in normDF.columns:
            if type(x[j]) == list:
                if len(x[j]) == 1:
                    normDF[j][i] = x[j][0]
                elif j == 'parent_mol':
                    normDF[j][i] = x[j][0]   
                elif j == 'Activity':
                    normDF['Activity'][i] = np.min(np.array(x['Activity']).astype(np.int))
                elif j == 'anntype':
                    if 'Negative' in normDF['anntype'][i]:
                        normDF['anntype'][i] = 'Negative'
                    else:
                        if 'Confirmed' in normDF['anntype'][i]:
                            normDF['anntype'][i] = 'Confirmed'               
            else:
                continue
                
    checkdupDF = normDF[normDF[colname].isin(duplist)]
    
    print('Original Dataset = '+ str(len(df)))
    print('Total number of duplicates = '+ str(len(duplist)))
    print('Normalized Dataset with no duplicates = '+ str(len(normDF)))
    print ('Inditex compounds with Negative  = '+ str(len(normDF[normDF['anntype']=='Negative'])))
    print ('Inditex compounds with Confirmed  = '+ str(len(normDF[normDF['anntype']=='Confirmed'])))
    print ('Inditex compounds with No information  = '+ str(len(normDF[normDF['anntype']=='No information'])))
    print('Final Normalized + Duplicates =' + str(len(normDF)+len(duplist)))
    
    return (normDF, checkdupDF)

In [ ]:
def chemFirstMD(df, smilesCol, endpoint):
    '''
    df = CarcNoinfoDF.copy()
    smilesCol = 'smiles'
    endpoint = 'inditexCarc'
    '''
    ##1D conversion
    PandasTools.AddMoleculeColumnToFrame(df,  smilesCol=smilesCol, molCol='mol1D')
    print ('compounds After 1D conversion  = '+ str(len(df)))
    ##2D conversion
    df['mol2D'] = df.mol1D.apply(lambda x: mol1Dto2Drdkit(x))
    print ('compounds After 2D conversion  = '+ str(len(df)))
    df = df.reset_index(drop=True)
    ##Standardised
    incDF, excDF = getStdMol(df, 'mol2D', endpoint)
    
    incDF = incDF.drop(columns=['mol1D','mol2D', 'phiID'])
    incDF = incDF.reset_index(drop=True)
    
    excDF = excDF.drop(columns=['mol1D','mol2D','phiID'])
    excDF = excDF.reset_index(drop=True)
    
    return (incDF, excDF)
    

In [ ]:
def chemSecondMD(df, endpoint, dupcheckcol, molcol, vpath, pH):
    '''
    df = incDF
    endpoint= 'inditex_carcinogen'
    dupcheckcol = 'parent_inkey'
    molcol= 'parent_mol'
    vpath = os.getcwd() ## current directory path
    pH = '7.4'
    
    '''
    
    ## checking duplicates
    normDF, checkdupDF = curationCMRDuplicates(df, dupcheckcol)
    ## Creates 2D SDfile in the folder:
    '\033[1m' + '\n2. EXCLUDED duplicates checking section:'+'\033[0m'
    print ('\033[1m' + '\n2.  Writing 2D SDFile'+'\033[0m')
    dir2Dname = '2-2Dcoord'
    createDir(vpath, dir2Dname)
    
    ## Writing 2D SDFile
    out2Dfile = vpath+'/'+dir2Dname+'/'+endpoint+'-2D-norm.sdf'
    props = list(normDF.columns)
    print ('Total 2D molecules : '+str(len(normDF)))
    writeSDFfromPandasDF(normDF, out2Dfile, molcol, props)
    
    ## Creates 3D SDfile in the folder:
    print ('\033[1m' + '\n3.  Writing 3D SDFile'+'\033[0m')
    dir3Dname = '3-3Dcoord' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dir3Dname)
    
    ## Writing 3D SDfile
    in2Dfile =  vpath+'/'+dir2Dname+'/'+endpoint+'-2D-norm.sdf'
    out3Dfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-3D-norm.sdf'
    convertTo3DwithCORINA(in2Dfile,out3Dfile)
    print ('Total 3D molecules : ' + str(len(PandasTools.LoadSDF(out3Dfile))))
    
    ## Creates protonation SDFile:
    print ('\033[1m' + '\n4.  Writing protonated SDFile at pH '+pH+'\033[0m')
    dirProtname = '4-3D-protonated' ## name directory to save 3D sdFile with 3D coordinates
    createDir(vpath, dirProtname)
    
    ## Writing protonated SDFile
    
    inProtfile =  vpath+'/'+dir3Dname+'/'+endpoint+'-3D-norm.sdf'
    outProtfile =  vpath+'/'+dirProtname+'/'+endpoint+'-3D-prot-norm.sdf'
    protonateWithMoka(inProtfile, pH, outProtfile)
    print ('Total prot molecules : ' + str(len(PandasTools.LoadSDF(outProtfile))))
    
    ##Writing an SDFile with protonation state field incorporated:
    print ('\033[1m' + '\n5.  Writing protonated SDFile with protonation state info'+'\033[0m')
    
    inStatefile = outProtfile
    protStateDF = protonationState(inStatefile, False)
    outStatefile = vpath+'/'+dirProtname+'/'+endpoint+'-norm-3D-prot-state.sdf'
    molcol = 'mol3Dprot'
    props = list(protStateDF.columns)
    writeSDFfromPandasDF(protStateDF, outStatefile, molcol, props)
    
    ## Calculating morgan fingerprints:
    print ('\033[1m' + '\n6.  Calculating Morgan Fingerprints, Running PCA 3 components'+'\033[0m')
    protStateDF['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(protStateDF))] ## giving phiID 
    res = calc_fp_arr(protStateDF.mol3Dprot, 4) ## calculating descriptors morgan fp
    pcaDF = getPCAscores(res, 3) ## runnning pca of 3 components
    protStateDF = protStateDF.join(pcaDF)
    print ('Total prot  molecules plus PCA scores : ' + str(len(protStateDF)))
    print ('\033[1m' + '\nFinished'+'\033[0m')
    
    return (protStateDF)

In [ ]:
def chemThirdMD(endpoint,refFile,refcolname,refcolACT, mdFile, mdcolname, mdcolACT):
    
    # REFERENCE PART
    
    ## Loading SDfile into pandas DF:
    
    refDF = PandasTools.LoadSDF(refFile, molColName='mol3Dprot')
    refDF.drop(columns=['ID'])
    
    ## Calculating morgan fingerprints:
    
    print ('\033[1m' + '\n1. REFERENCE: Calculating Morgan Fingerprints, Running PCA 3 components'+'\033[0m')
    refDF['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(refDF))] ## giving phiID 
    res = calc_fp_arr(refDF.mol3Dprot, 4) ## calculating descriptors morgan fp
    pcaDF = getPCAscores(res, 3) ## runnning pca of 3 components
    refDF = refDF.join(pcaDF)
    
    ## Reference parameters section:
    
    refDC1 = 'PC1'     ## column name first descriptor e.g. PC1, LogP, ...
    refDC2 = 'PC2'     ## column name second descriptor e.g. PC1, LogP, ...
    refmolcol = 'mol3Dprot'  ## molecule column 
    refName = refcolname   ## name column
    refACT = refcolACT    ## name of the activity field 
    refimg = endpoint+'-refimg/' ## Reference images folder path
    
    # MODEL PART
    
    ## Loading SDfile into pandas DF:
    
    mdDF = PandasTools.LoadSDF(mdFile, molColName='mol3Dprot')
    mdDF.drop(columns=['ID'])
    
    ## Calculating morgan fingerprints:
    
    print ('\033[1m' + '\n2. MODEL: Calculating Morgan Fingerprints, Running PCA 3 components'+'\033[0m')
    mdDF['phiID'] = [str('mol%0.6d'%(int(x)+1)) for x in range(len(mdDF))] ## giving phiID 
    res = calc_fp_arr(mdDF.mol3Dprot, 4) ## calculating descriptors morgan fp
    pcaDF = getPCAscores(res, 3) ## runnning pca of 3 components
    mdDF = mdDF.join(pcaDF)

    ## Model parameters section:
    mdDC1 = 'PC1'     ## column name first descriptor e.g. PC1, LogP, ...
    mdDC2 = 'PC2'     ## column name second descriptor e.g. PC1, LogP, ...
    mdmolcol = 'mol3Dprot'  ## molecule column 
    mdName = mdcolname   ## name column
    mdACT = mdcolACT    ## name of the activity field 
    mdimg = endpoint+'-mdimg/'  ## Model images folder path
    
    
    ### Bokeh interactive graph section

    scatter_mol_RefvsMD(refDF, refDC1, refDC2, refmolcol, refName, refACT, refimg, 
                        mdDF, mdDC1, mdDC2, mdmolcol, mdName, mdACT, mdimg)
    
    return (refDF, mdDF)
    

## 9. FOUR FOLD graph

In [ ]:
def FourfoldDisplay(TP, TN, FP, FN, label, name, endpoint):
    """ Draws confusion matrix graphical representaion

    """
    print (TP, TN, FP, FN)
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    print ('sens', sensitivity, 'spec', specificity)
    width = np.pi / 2.0
    theta = np.radians([0,90,180,270])
    table = [FP,TP,FN,TN]
##    plt.figure("RF-Qualitative_validation")
    plt.figure()
    plt.clf()
    ax = plt.subplot(121, polar=True, adjustable='box', aspect=2)    
    bars = ax.bar(theta, table, width=width, color=["red", "lightblue", "red", "lightblue"])
##    plt.title( label )

    ax.set_xticklabels(["","FP (%s) \n\n" % str(int(FP)), "",  "TP (%s) \n\n" % str(int(TP)), "", "\n\n\nFN (%s)" % str(int(FN)), 
                        "",  "\n\n\nTN (%s)" % str(int(TN))], fontsize=14)
    ax.set_yticks([])
    ax.grid(False)
    ax.axes.spines['polar'].set_visible(False)
    
    ax2 = plt.subplot(122, adjustable='box', aspect=4)
    plt.ylim([0,1])
##    plt.title(endpoint+'\n')
    
    bar_width = 0.5
    y = [0, sensitivity, specificity, 0]
    index = np.arange(4)
    ax2.bar(index, y, bar_width, color=["lightgreen","lightgrey"])
    #ax.offset(0.5)
    plt.xticks( index + bar_width / 2.0, ("", 'Sens', 'Spec', ""))
    
    plt.savefig(name)


## 10. Histogram plot plus adding category column to Dataframe

In [ ]:
def categorizeByhist(df, activity, bins):
    
    '''
    
    This function returns a dataframe with a histogram category column. It creates a folder 
    called histogram, and inside it creates the histogram plot. 
    
    Input parameters:
    
    df = protDF.copy()                   #### dataframe to be used
    activity = 'quantitative_activity'   #### activity name column 
    bins = 10                            #### int number of bins to be histogrammed
    
    e.g. df = histogramTOLabels(df, activity, bins)
    
    '''
    xdf = df.copy()
    xdf[activity] = np.around(xdf[activity], 4) ## round to 4 decimals
    hist = xdf[activity].value_counts(bins=bins).sort_index()
    print (hist)
    
    my_dpi = 96
    fig = plt.figure(figsize=(1250/my_dpi, 800/my_dpi))
    myplot = sns.barplot(hist.index, hist.values, alpha=0.8)
    for item in myplot.get_xticklabels():
        item.set_rotation(75)
    plt.title(activity+' histogram')
    plt.ylabel('Number of Compounds', fontsize=12)
    plt.xlabel(activity, fontsize=12)
    plt.show()
    
    directory = 'histogram'

    if not os.path.exists(directory):
        os.makedirs(directory)
        print (directory + ' is created')
    else: 
        print(directory + ' already exists')
    
    fig.savefig(directory+'/histogram_'+str(bins)+'.png')
    
    xdf['histcat'] = pd.cut(xdf[activity], bins=bins, right=True, labels=[str(x) for x in range(bins)])

    return (xdf)    